# import 

In [1]:
import torch
import torch.nn.functional as F
import torch.utils.data as Data
from torch.autograd import Variable
import pandas as pd
import numpy as np
import time
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import mutual_info_classif
import datetime
import pickle
import scipy.sparse as ss
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['NUMEXPR_MAX_THREADS'] = '32'
# import seaborn as sns

import IPython.display as ipd
import copy
import random
from pandarallel import pandarallel
# Initialization
pandarallel.initialize(progress_bar=True)
# df.parallel_apply(func)
import time
from gensim.models.word2vec import Word2Vec 
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold,KFold

from transformers import *
import torch.nn as nn
import math
from tqdm import tqdm_notebook as tqdm
from transformers.modeling_bert import BertConfig, BertEncoder, BertAttention,\
BertSelfAttention,BertLayer,BertPooler,BertLayerNorm

from gensim.models.word2vec import Word2Vec 
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold,KFold
from scipy.special import softmax

from category_encoders import *

2021-06-27 17:17:05,570 - INFO - PyTorch version 1.1.0 available.


INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/huangweilin/anaconda3/envs/hyr/lib

# Read

In [2]:
logging.info('start read')
df_master_records = pickle.load(open('../data_sortout/df_master_records.pickle', 'rb'))
se_id_install_list = pickle.load(open('../data_sortout/se_id_install_list.pickle', 'rb'))
df_install_behave = pickle.load(open('../data_sortout/df_install_behave_no_date.pickle', 'rb'))
df_behave_time = pickle.load(open('../data_sortout/df_time_cut.pickle', 'rb'))
# df_userlog = pickle.load(open('../data_sortout/df_userlog_sequence_less.pickle', 'rb'))
se_userlog_cross = pickle.load(open('../data_sortout/se_userlog_cross_id.pickle', 'rb'))
df_userlog_time_seq = pickle.load(open('../data_sortout/df_userlog_time_seq.pickle', 'rb'))

# df_app_list_te_sequence = pickle.load(open('../data_sortout/df_app_list_target_encode_sequence.pickle', 'rb'))
# df_app_behave_te_sequence = pickle.load(open('../data_sortout/df_app_behave_target_encode_sequence.pickle', 'rb'))

df_app_list_te_qcut = pickle.load(open('../data_sortout/df_app_list_target_qcut.pickle', 'rb'))
df_app_behave_te_qcut = pickle.load(open('../data_sortout/df_app_behave_target_qcut.pickle', 'rb'))


logging.info('finish read')

2021-06-27 17:17:10,718 - INFO - start read
2021-06-27 17:18:02,146 - INFO - finish read


In [3]:
min(df_master_records['loan_date']), max(df_master_records['loan_date']), 

(Timestamp('2019-01-01 00:00:00'), Timestamp('2019-09-29 00:00:00'))

In [4]:
df_master_records.shape, se_id_install_list.shape, df_install_behave.shape, se_userlog_cross.shape

((402840, 21), (178608,), (197737, 2), (319977,))

In [5]:
print(len(set(se_id_install_list.index) & set(df_install_behave.index) & set(se_userlog_cross.index)), \
len(set(se_id_install_list.index) & set(df_install_behave.index) )      ,\
len(set(se_id_install_list.index) & set(se_userlog_cross.index))      ,\
len(set(df_install_behave.index) & set(se_userlog_cross.index))    )  


127213 157497 143457 159722


# 数据划分

In [6]:
split_date = datetime.datetime(2019, 8, 31)
end_date = datetime.datetime(2019, 9, 30)

df_master_records = df_master_records.dropna(axis=0, how='any')
df_train_master = df_master_records.query('loan_date <= @split_date')
df_test_master = df_master_records.query('loan_date > @split_date & loan_date <= @end_date')
all_train_id = list(df_train_master.index)
all_test_id = list(df_test_master.index)
logging.info('all_train_id len :%d, all_test_id: %d' % (len(all_train_id), len(all_test_id)))
df_target = df_master_records[['target_1m30+', 'target_2m30+', 'target_3m30+', 'target_4m30+']]

2021-06-27 17:18:04,289 - INFO - all_train_id len :326082, all_test_id: 75896


In [7]:
max_app_list_id = max(se_id_install_list.apply(max))
max_app_behave_id = max(df_install_behave['pkg_id'].apply(max))
max_uselog_id = max(se_userlog_cross.apply(max))
start_app_list_id = max_app_list_id + 1
start_app_behave_id = max_app_behave_id + 1 
start_uselog_id = max_uselog_id + 1

# Feature

## user info

In [8]:
def get_master_user_discrete(df_master_records):
    
    df_master_records['qcut_amount_bin'] = pd.qcut(df_master_records['amount_bin'], 5)
    df_master_records['new_client'] = df_master_records['loan_sequence'] == 1
    df_master_records['qcut_age'] = pd.qcut(df_master_records['age'], 5, duplicates='drop')
    
    df_master_records['qcut_min_income'] = pd.qcut(df_master_records['min_income'], 6, duplicates='drop')
    df_master_records['qcut_max_income'] = pd.qcut(df_master_records['max_income'].apply(int), 6, duplicates='drop')

#     df_master_records['qcut_loan_sequence'] = pd.qcut(df_master_records['loan_sequence'], 6, duplicates='drop')
#     pne_hot_cols = ['months', 'gender', 'educationid', 'marriagestatusid', 'income', 
#                     'qcut_amount_bin', 'new_client', 'qcut_loan_sequence', 'qcut_age', 'qcut_min_income', 'qcut_max_income']
    pne_hot_cols = ['months', 'gender', 'educationid', 'marriagestatusid', 'income', 
                'qcut_amount_bin', 'qcut_age', 'qcut_min_income', 'qcut_max_income']

    return  pd.get_dummies(df_master_records[pne_hot_cols], columns = pne_hot_cols)
df_user_one_hot = get_master_user_discrete(df_master_records)

In [9]:
df_user_one_hot.shape

(401978, 102)

## target encode

### tools

In [10]:
def target_encode_cross_validation(df_train, df_test, se_target_col, random_seed = 0):
    
    n_flod = 5
    folds = KFold(n_splits=n_flod, shuffle=True, random_state=random_seed)
        
    df_vals = []
    df_test_ret = df_test[['pkg_id']].copy()
    df_test_ret[['pkg_id']] = 0
    
    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(df_train, se_target_col)):
        
        target_encoder = TargetEncoder(cols=['pkg_id'])
        train_df = df_train.iloc[trn_idx]
        val_df = df_train.iloc[val_idx]
        target_encoder.fit(train_df['pkg_id'], se_target_col.iloc[trn_idx])
        
        df_vals.append(target_encoder.transform(val_df['pkg_id']))  
        
        df_test_ret[['pkg_id']] += target_encoder.transform(df_test['pkg_id'])[['pkg_id']] / n_flod
        
        logging.info("finish fold : %d" % n_fold)    
    
    df_val_ret = pd.concat(df_vals)
    df_val_ret = df_val_ret.sort_index()
    
    df_val_ret.columns = se_target_col.name + '_' + df_val_ret.columns
    df_test_ret.columns = se_target_col.name + '_' + df_test_ret.columns
    return df_val_ret, df_test_ret

def target_sort_out(se_data):
    master_ids = []
    pkg_ids = []
    for master_id in se_data.index:
        pkg_ids.extend(list(se_data.loc[master_id]))
        master_ids.extend([master_id] * len(se_data.loc[master_id]))
    
    return pd.DataFrame({
        'master_id' : master_ids,
        'pkg_id' : pkg_ids,
    })


### app list

In [9]:
df_app_list_target_id = target_sort_out(se_id_install_list)
df_app_list_target_id = pd.merge(df_app_list_target_id, df_target, how = 'left', left_on = 'master_id', right_index = True)
df_app_list_target_id = df_app_list_target_id.dropna()

KeyboardInterrupt: 

Exception ignored in: 'pandas._libs.lib.c_is_list_like'
Traceback (most recent call last):
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/abc.py", line 180, in __instancecheck__
    def __instancecheck__(cls, instance):
KeyboardInterrupt
2020-09-04 12:55:02,517 - ERROR - Internal Python error in the inspect module.
Below is the traceback from this internal error.

2020-09-04 12:55:02,521 - INFO - 
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-45fd27fb4602>", line 1, in <module>
    df_app_list_target_id = target_sort_out(se_id_install_list)
  File "<ipython-input-8-9be4276a0b8a>", line 35, in target_sort_out
    master_ids.extend([master_id] * len(se_data.loc[master_id]))
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/pandas/core/indexing.py", line 1768, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/pandas/core/indexing.py", line 1964, in _getitem_axis
    self._validate_key(key, axis)
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/pandas/core/indexing.py", line 1827, in _validate_key
    if com.is_bool_indexer(key):
  File "/home/huan

KeyboardInterrupt: 

In [ ]:
df_train_app_list = df_app_list_target_id[df_app_list_target_id.master_id.isin(set(all_train_id))]
df_test_app_list = df_app_list_target_id[df_app_list_target_id.master_id.isin(set(all_test_id))]

In [ ]:
df_app_list_train_target_encode_list, df_app_list_test_target_encode_list = [], []
for target in ['target_1m30+', 'target_2m30+', 'target_3m30+', 'target_4m30+']:
    df_app_list_train_target_encode, df_app_list_test_target_encode = target_encode_cross_validation(df_train_app_list, df_test_app_list, df_train_app_list[target])
    df_app_list_train_target_encode_list.append(df_app_list_train_target_encode)
    df_app_list_test_target_encode_list.append(df_app_list_test_target_encode)

In [13]:
df_app_list_train_target_encode = pd.concat(df_app_list_train_target_encode_list, axis = 1)
df_app_list_test_target_encode = pd.concat(df_app_list_test_target_encode_list, axis = 1)
df_app_list_target_encode = df_app_list_train_target_encode.append(df_app_list_test_target_encode).sort_index()

In [15]:
df_app_list_target_encode_with_id = pd.concat([df_app_list_target_id, df_app_list_target_encode], axis = 1)

In [18]:
se_target_qcut_list = []
for target_name in ['target_1m30+_pkg_id', 'target_2m30+_pkg_id', 'target_3m30+_pkg_id', 'target_4m30+_pkg_id']:
    se_target_qcut = pd.qcut(df_app_list_target_encode[target_name], 16)
    se_target_qcut_list.append(se_target_qcut)

In [19]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [20]:
df_target_qcut = pd.concat(se_target_qcut_list, axis = 1)
df_target_qcut['target_1m30_qcut'] = preprocessing.LabelEncoder().fit_transform(df_target_qcut['target_1m30+_pkg_id']) + 1
df_target_qcut['target_2m30_qcut'] = preprocessing.LabelEncoder().fit_transform(df_target_qcut['target_2m30+_pkg_id']) + 1
df_target_qcut['target_3m30_qcut'] = preprocessing.LabelEncoder().fit_transform(df_target_qcut['target_3m30+_pkg_id']) + 1
df_target_qcut['target_4m30_qcut'] = preprocessing.LabelEncoder().fit_transform(df_target_qcut['target_4m30+_pkg_id']) + 1

In [21]:
df_app_list_target_qcut_with_id = pd.concat([df_app_list_target_id[['master_id', 'pkg_id']], df_target_qcut[['target_1m30_qcut', 'target_2m30_qcut', 'target_3m30_qcut', 'target_4m30_qcut',]]], axis = 1)

In [22]:
df_app_list_target_encode_sequence = df_app_list_target_qcut_with_id.groupby('master_id').apply(lambda x : pd.Series({
    'target_1m30_qcut' : x['target_1m30_qcut'].values,
    'target_2m30_qcut' : x['target_2m30_qcut'].values,
    'target_3m30_qcut' : x['target_3m30_qcut'].values,
    'target_4m30_qcut' : x['target_4m30_qcut'].values,
    'pkg_id' : x['pkg_id'].values,
}))
# pickle.dump(df_app_list_target_encode_sequence, open('../data_sortout/df_app_list_target_qcut.pickle', 'wb'))

In [34]:
check_id = 15161
df_app_list_target_encode_sequence['pkg_id'].loc[check_id] == se_id_install_list.loc[check_id]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [14]:
df_app_list_target_encode_with_id = pd.concat([df_app_list_target_id, df_app_list_target_encode], axis = 1)
df_app_list_target_encode_sequence = df_app_list_target_encode_with_id.groupby('master_id').apply(lambda x : pd.Series({
    'target_1m30+_pkg_id' : x['target_1m30+_pkg_id'].values,
    'target_2m30+_pkg_id' : x['target_2m30+_pkg_id'].values,
    'target_3m30+_pkg_id' : x['target_3m30+_pkg_id'].values,
    'target_4m30+_pkg_id' : x['target_4m30+_pkg_id'].values,
}))
pickle.dump(df_app_list_target_encode_sequence, open('../data_sortout/df_app_list_target_encode_sequence.pickle', 'wb'))

### app behave

In [9]:
df_app_behave_target_id = target_sort_out(df_install_behave['pkg_id'])
df_app_behave_target_id = pd.merge(df_app_behave_target_id, df_target, how = 'left', left_on = 'master_id', right_index = True)
df_app_behave_target_id = df_app_behave_target_id.dropna()

In [10]:
df_train_app_behave = df_app_behave_target_id[df_app_behave_target_id.master_id.isin(set(all_train_id))]
df_test_app_behave = df_app_behave_target_id[df_app_behave_target_id.master_id.isin(set(all_test_id))]

In [11]:
df_app_behave_train_target_encode_list, df_app_behave_test_target_encode_list = [], []
for target in ['target_1m30+', 'target_2m30+', 'target_3m30+', 'target_4m30+']:
    df_app_behave_train_target_encode, df_app_behave_test_target_encode = target_encode_cross_validation(df_train_app_behave, df_test_app_behave, df_train_app_behave[target])
    df_app_behave_train_target_encode_list.append(df_app_behave_train_target_encode)
    df_app_behave_test_target_encode_list.append(df_app_behave_test_target_encode)

2020-09-01 17:59:01,617 - INFO - finish fold : 0
2020-09-01 17:59:16,364 - INFO - finish fold : 1
2020-09-01 17:59:31,026 - INFO - finish fold : 2
2020-09-01 17:59:45,828 - INFO - finish fold : 3
2020-09-01 18:00:00,482 - INFO - finish fold : 4
2020-09-01 18:00:17,547 - INFO - finish fold : 0
2020-09-01 18:00:32,083 - INFO - finish fold : 1
2020-09-01 18:00:46,738 - INFO - finish fold : 2
2020-09-01 18:01:01,456 - INFO - finish fold : 3
2020-09-01 18:01:16,037 - INFO - finish fold : 4
2020-09-01 18:01:33,180 - INFO - finish fold : 0
2020-09-01 18:01:47,794 - INFO - finish fold : 1
2020-09-01 18:02:02,378 - INFO - finish fold : 2
2020-09-01 18:02:17,132 - INFO - finish fold : 3
2020-09-01 18:02:31,752 - INFO - finish fold : 4
2020-09-01 18:02:48,824 - INFO - finish fold : 0
2020-09-01 18:03:03,496 - INFO - finish fold : 1
2020-09-01 18:03:18,211 - INFO - finish fold : 2
2020-09-01 18:03:32,846 - INFO - finish fold : 3
2020-09-01 18:03:47,447 - INFO - finish fold : 4


In [12]:
df_app_behave_train_target_encode = pd.concat(df_app_behave_train_target_encode_list, axis = 1)
df_app_behave_test_target_encode = pd.concat(df_app_behave_test_target_encode_list, axis = 1)
df_app_behave_target_encode = df_app_behave_train_target_encode.append(df_app_behave_test_target_encode).sort_index()

In [14]:
se_target_qcut_list = []
for target_name in ['target_1m30+_pkg_id', 'target_2m30+_pkg_id', 'target_3m30+_pkg_id', 'target_4m30+_pkg_id']:
    se_target_qcut = pd.qcut(df_app_behave_target_encode[target_name], 16)
    se_target_qcut_list.append(se_target_qcut)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

df_target_qcut = pd.concat(se_target_qcut_list, axis = 1)
df_target_qcut['target_1m30_qcut'] = preprocessing.LabelEncoder().fit_transform(df_target_qcut['target_1m30+_pkg_id']) + 1
df_target_qcut['target_2m30_qcut'] = preprocessing.LabelEncoder().fit_transform(df_target_qcut['target_2m30+_pkg_id']) + 1
df_target_qcut['target_3m30_qcut'] = preprocessing.LabelEncoder().fit_transform(df_target_qcut['target_3m30+_pkg_id']) + 1
df_target_qcut['target_4m30_qcut'] = preprocessing.LabelEncoder().fit_transform(df_target_qcut['target_4m30+_pkg_id']) + 1

In [15]:
df_app_behave_target_qcut_with_id = pd.concat([df_app_behave_target_id[['master_id', 'pkg_id']], df_target_qcut[['target_1m30_qcut', 'target_2m30_qcut', 'target_3m30_qcut', 'target_4m30_qcut',]]], axis = 1)
df_app_behave_target_encode_sequence = df_app_behave_target_qcut_with_id.groupby('master_id').apply(lambda x : pd.Series({
    'target_1m30_qcut' : x['target_1m30_qcut'].values,
    'target_2m30_qcut' : x['target_2m30_qcut'].values,
    'target_3m30_qcut' : x['target_3m30_qcut'].values,
    'target_4m30_qcut' : x['target_4m30_qcut'].values,
}))
pickle.dump(df_app_behave_target_encode_sequence, open('../data_sortout/df_app_behave_target_qcut.pickle', 'wb'))

In [20]:
df_app_behave_target_encode_with_id = pd.concat([df_app_behave_target_id, df_app_behave_target_encode], axis = 1)

In [21]:
df_app_behave_target_encode_sequence = df_app_behave_target_encode_with_id.groupby('master_id').apply(lambda x : pd.Series({
    'target_1m30+_pkg_id' : x['target_1m30+_pkg_id'].values,
    'target_2m30+_pkg_id' : x['target_2m30+_pkg_id'].values,
    'target_3m30+_pkg_id' : x['target_3m30+_pkg_id'].values,
    'target_4m30+_pkg_id' : x['target_4m30+_pkg_id'].values,
}))

In [22]:
# df_app_behave_target_encode_sequence

In [23]:
pickle.dump(df_app_behave_target_encode_sequence, open('../data_sortout/df_app_behave_target_encode_sequence.pickle', 'wb'))

# word2vec

## app list

In [10]:
sentences = []
for data_list in tqdm(se_id_install_list):
    sentence = [str(start_app_list_id),]
    for data in data_list:
        sentence.append(str(data))
    sentences.append(sentence)
window = 200
mincount = 1
n_dim = 100
logging.info('start word2vec')
wv_model= Word2Vec(sentences, min_count=mincount, size = n_dim, window = window, sg = 0, workers = 12)
logging.info('finish word2vec')
pickle.dump(wv_model, open('../data_sortout/wv_model_app_list.pickle', 'wb'))

2021-06-27 15:11:04,708 - INFO - start word2vec
2021-06-27 15:11:04,710 - INFO - collecting all words and their counts
2021-06-27 15:11:04,711 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-06-27 15:11:04,789 - INFO - PROGRESS: at sentence #10000, processed 485059 words, keeping 26159 word types
2021-06-27 15:11:04,868 - INFO - PROGRESS: at sentence #20000, processed 973429 words, keeping 39080 word types


2021-06-27 15:11:04,947 - INFO - PROGRESS: at sentence #30000, processed 1455010 words, keeping 48144 word types
2021-06-27 15:11:05,027 - INFO - PROGRESS: at sentence #40000, processed 1938752 words, keeping 55663 word types
2021-06-27 15:11:05,106 - INFO - PROGRESS: at sentence #50000, processed 2417939 words, keeping 61823 word types
2021-06-27 15:11:05,185 - INFO - PROGRESS: at sentence #60000, processed 2899438 words, keeping 67422 word types
2021-06-27 15:11:05,285 - INFO - PROGRESS: at sentence #70000, processed 3381230 words, keeping 71996 word types
2021-06-27 15:11:05,365 - INFO - PROGRESS: at sentence #80000, processed 3863909 words, keeping 76416 word types
2021-06-27 15:11:05,445 - INFO - PROGRESS: at sentence #90000, processed 4346541 words, keeping 81555 word types
2021-06-27 15:11:05,526 - INFO - PROGRESS: at sentence #100000, processed 4827439 words, keeping 86308 word types
2021-06-27 15:11:05,609 - INFO - PROGRESS: at sentence #110000, processed 5314069 words, keepin

2021-06-27 15:12:07,452 - INFO - worker thread finished; awaiting finish of 9 more threads
2021-06-27 15:12:07,475 - INFO - worker thread finished; awaiting finish of 8 more threads
2021-06-27 15:12:07,481 - INFO - worker thread finished; awaiting finish of 7 more threads
2021-06-27 15:12:07,484 - INFO - worker thread finished; awaiting finish of 6 more threads
2021-06-27 15:12:07,491 - INFO - worker thread finished; awaiting finish of 5 more threads
2021-06-27 15:12:07,494 - INFO - worker thread finished; awaiting finish of 4 more threads
2021-06-27 15:12:07,501 - INFO - worker thread finished; awaiting finish of 3 more threads
2021-06-27 15:12:07,507 - INFO - worker thread finished; awaiting finish of 2 more threads
2021-06-27 15:12:07,508 - INFO - worker thread finished; awaiting finish of 1 more threads
2021-06-27 15:12:07,509 - INFO - worker thread finished; awaiting finish of 0 more threads
2021-06-27 15:12:07,510 - INFO - EPOCH - 3 : training on 8739735 raw words (6774307 effect

In [ ]:
# se_app_list_embedding = se_id_install_list.apply(lambda x : wv_model.wv[x].sum(axis = 0)) 
# def apply_wv_sum(app_list):
#     ret = np.zeros(100)
#     for app in app_list:
#         if(app in wv_model.wv):
#             ret += wv_model.wv[app]
#     return ret

# se_app_list_embedding = se_id_install_list.apply(apply_wv_sum) 
# se_app_list_embedding = pickle.load(open('/data/ccnth/ppd/se_app_list_embedding.pickle', 'rb'))

## app behave

In [11]:
sentences = []
for data_list in tqdm(df_install_behave['pkg_id']):
    sentence = [str(start_app_behave_id),]
    for data in data_list:
        sentence.append(str(data))
    sentences.append(sentence)
window = 5
mincount = 1
n_dim = 100
logging.info('start word2vec install behave')
wv_model= Word2Vec(sentences, min_count=mincount, size = n_dim, window = window, sg = 0, workers = 12)
logging.info('finish word2vec install behave')
pickle.dump(wv_model, open('../data_sortout/wv_model_app_behave.pickle', 'wb'))

2021-06-27 15:12:45,796 - INFO - start word2vec install behave
2021-06-27 15:12:45,798 - INFO - collecting all words and their counts
2021-06-27 15:12:45,799 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


2021-06-27 15:12:46,004 - INFO - PROGRESS: at sentence #10000, processed 1122728 words, keeping 46826 word types
2021-06-27 15:12:46,212 - INFO - PROGRESS: at sentence #20000, processed 2288382 words, keeping 68779 word types
2021-06-27 15:12:46,429 - INFO - PROGRESS: at sentence #30000, processed 3509631 words, keeping 86252 word types
2021-06-27 15:12:46,672 - INFO - PROGRESS: at sentence #40000, processed 4745747 words, keeping 98374 word types
2021-06-27 15:12:46,901 - INFO - PROGRESS: at sentence #50000, processed 6087544 words, keeping 109501 word types
2021-06-27 15:12:47,110 - INFO - PROGRESS: at sentence #60000, processed 7292233 words, keeping 118388 word types
2021-06-27 15:12:47,358 - INFO - PROGRESS: at sentence #70000, processed 8748442 words, keeping 127558 word types
2021-06-27 15:12:47,596 - INFO - PROGRESS: at sentence #80000, processed 9935970 words, keeping 134582 word types
2021-06-27 15:12:47,854 - INFO - PROGRESS: at sentence #90000, processed 11322867 words, kee

2021-06-27 15:14:17,368 - INFO - EPOCH 2 - PROGRESS: at 95.77% examples, 1075600 words/s, in_qsize 24, out_qsize 1
2021-06-27 15:14:18,130 - INFO - worker thread finished; awaiting finish of 11 more threads
2021-06-27 15:14:18,139 - INFO - worker thread finished; awaiting finish of 10 more threads
2021-06-27 15:14:18,145 - INFO - worker thread finished; awaiting finish of 9 more threads
2021-06-27 15:14:18,153 - INFO - worker thread finished; awaiting finish of 8 more threads
2021-06-27 15:14:18,156 - INFO - worker thread finished; awaiting finish of 7 more threads
2021-06-27 15:14:18,166 - INFO - worker thread finished; awaiting finish of 6 more threads
2021-06-27 15:14:18,180 - INFO - worker thread finished; awaiting finish of 5 more threads
2021-06-27 15:14:18,184 - INFO - worker thread finished; awaiting finish of 4 more threads
2021-06-27 15:14:18,190 - INFO - worker thread finished; awaiting finish of 3 more threads
2021-06-27 15:14:18,193 - INFO - worker thread finished; awaitin

Traceback (most recent call last):
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-cf484b119adc>", line 11, in <module>
    wv_model= Word2Vec(sentences, min_count=mincount, size = n_dim, window = window, sg = 0, workers = 12)
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/gensim/models/word2vec.py", line 783, in __init__
    fast_version=FAST_VERSION)
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/gensim/models/base_any2vec.py", line 763, in __init__
    end_alpha=self.min_alpha, compute_loss=compute_loss)
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/gensim/models/word2vec.py", line 910, in train
    queue_factor=queue_factor, report_delay=report_delay, compute_loss=compute_loss, callbacks=callbacks)
  File "/home/huangweilin/anaconda3/envs/hyr/l

KeyboardInterrupt: 

In [ ]:
# logging.info('se_app_behave_embedding')
# def get_behave_embed(data_list):
#     data_list_str = [str(data) for data in data_list]
    
#     return wv_model_app_behave.wv[data_list_str].sum(axis = 0)
# se_app_behave_embedding = df_install_behave.pkg_id.apply(get_behave_embed) 
# logging.info('se_app_behave_embedding')
# wv_behave_sum = np.array(list(se_app_behave_embedding.values))
# x_mean = np.mean(wv_behave_sum, axis = 0)
# x_std = np.std(wv_behave_sum, axis = 0)
# wv_behave_sum_norm = (wv_behave_sum - x_mean) / x_std
# se_app_behave_embedding_norm = pd.Series(list(wv_behave_sum_norm))
# se_app_behave_embedding_norm.index = se_app_behave_embedding.index

## userlog

In [ ]:
sentences = []
for data_list in tqdm(se_userlog_cross):
    sentence = [str(start_uselog_id),]
    for data in data_list:
        sentence.append(str(data))
    sentences.append(sentence)
    
window = 5
mincount = 1
n_dim = 100
logging.info('start word2vec install behave')
wv_model= Word2Vec(sentences, min_count=mincount, size = n_dim, window = window, sg = 0, workers = 12)
logging.info('finish word2vec install behave')
pickle.dump(wv_model, open('../data_sortout/wv_model_userlog_cross.pickle', 'wb'))

## load

In [10]:
wv_model_app_list = pickle.load(open('../data_sortout/wv_model_app_list.pickle', 'rb'))
wv_model_app_behave = pickle.load(open('../data_sortout/wv_model_app_behave.pickle', 'rb'))
wv_model_userlog = pickle.load(open('../data_sortout/wv_model_userlog_cross.pickle', 'rb'))

# train

## 调参

In [11]:
from collections import namedtuple

ARG = namedtuple('ARG', [
    'batch_size',
    'epoch',
    'lr',
    'weight_decay',
    'debug',
    'n_embedding',
    'app_install_list_max_length',
    'app_behave_max_length',
    'userlog_max_length',
    'n_eval',
    'dropout_rate',
    'n_worker',
    'use_cuda',
    'n_gpu',
    'device',
    'card_list'
])
 
args = ARG(
    batch_size = 256,
    epoch = 12,
    lr = 0.001,
    weight_decay = 0.0,
    dropout_rate = 0.,
    debug = False,
    n_embedding = 100,
    app_install_list_max_length = 256,
    app_behave_max_length = 256,
    userlog_max_length = 256,
    n_eval = len(all_test_id)+1,
    n_worker = 0,
    use_cuda = True,
    n_gpu = 1,
    card_list = [0, 1],
    device=torch.device("cuda:1"),
#     device=torch.device("cpu")

)

## dataset

In [12]:
install_behave_set = set(df_install_behave.index) & (set(all_train_id) | set(all_test_id))
install_list_set = set(se_id_install_list.index) & (set(all_train_id) | set(all_test_id))
user_info_set = set(df_user_one_hot.index) & (set(all_train_id) | set(all_test_id))
user_log_set = set(se_userlog_cross.index) & (set(all_train_id) | set(all_test_id))

In [13]:
class AppDataset(Data.Dataset):
    def __init__(self, master_ids):
        self.master_ids = list(master_ids)
        
    def __len__(self):
        return len(self.master_ids)
    
    def __getitem__(self,idx):
        return self.master_ids[idx]

x_dict = {
    
    'user_info' : np.zeros((args.batch_size, df_user_one_hot.shape[1])),

    'app_list' : np.zeros((args.batch_size, args.app_install_list_max_length + 1)).astype('int'),
    'app_list_te_qcut' : np.zeros((args.batch_size, args.app_install_list_max_length + 1, 4)).astype('int'),
    'app_list_len' :  np.zeros((args.batch_size,)).astype('int'),
    
    'app_behave' : np.zeros((args.batch_size, args.app_behave_max_length + 1)).astype('int'),
    'app_behave_time_cut' : np.zeros((args.batch_size, args.app_behave_max_length + 1)).astype('int'),
    'app_behave_time_qcut' : np.zeros((args.batch_size, args.app_behave_max_length + 1)).astype('int'),
    
    'app_behave_action' : np.zeros((args.batch_size, args.app_behave_max_length + 1)).astype('int'),
    'app_behave_te_qcut' : np.zeros((args.batch_size, args.app_behave_max_length + 1, 4)).astype('int'),

    'app_behave_len' :  np.zeros((args.batch_size,)).astype('int'),
    
    'userlog' : np.zeros((args.batch_size, args.userlog_max_length + 1)).astype('int'),
    'userlog_len' :  np.zeros((args.batch_size,)).astype('int'),
    'userlog_day_qcut' : np.zeros((args.batch_size, args.userlog_max_length + 1)).astype('int'),
    'userlog_day_cut' : np.zeros((args.batch_size, args.userlog_max_length + 1)).astype('int'),
    'userlog_second_qcut' : np.zeros((args.batch_size, args.userlog_max_length + 1)).astype('int'),
    'userlog_second_cut' : np.zeros((args.batch_size, args.userlog_max_length + 1)).astype('int'),
    
    'view_mask' : np.zeros((args.batch_size, 4)).astype('int'),
}

def set_first_token():
    x_dict['app_list'][:, 0] = start_app_list_id
    x_dict['app_behave'][:, 0] = start_app_behave_id
    x_dict['userlog'][:, 0] = start_uselog_id

    x_dict['app_list'][:, 0] = 0
    x_dict['app_behave'][:, 0] = 0
    x_dict['userlog'][:, 0] = 0

    x_dict['userlog_day_qcut'][:, 0] = 8
    x_dict['userlog_day_cut'][:, 0] = 8
    x_dict['userlog_second_qcut'][:, 0] = 32
    x_dict['userlog_second_cut'][:, 0] = 32

set_first_token()

def collate_fn(master_ids):
    master_ids = np.array(master_ids)

#     sub_master_id = se_id_install_list.loc[master_ids]
#     df_sub_behave = df_install_behave.loc[master_ids]
#     df_sub_time = df_behave_time.loc[master_ids]
    for i, master_id in enumerate(master_ids):
        if master_id in user_info_set:
            x_dict['user_info'][i] = df_user_one_hot.loc[master_id].values
            x_dict['view_mask'][i][0] = 1
        else:
            x_dict['user_info'][i] = 0
            x_dict['view_mask'][i][0] = 0

        if master_id in install_list_set:
            app_list = se_id_install_list.at[master_id][:args.app_install_list_max_length]
            x_dict['app_list_len'][i] = len(app_list) + 1
            x_dict['app_list'][i][1 : x_dict['app_list_len'][i]] = app_list
            x_dict['app_list'][i][x_dict['app_list_len'][i] :] = 0
            
#             target_encode_data = np.array(list(df_app_list_te_qcut.loc[master_id].values))[:, :args.app_install_list_max_length].T
#             x_dict['app_list_te_qcut'][i][1 : x_dict['app_list_len'][i]] = target_encode_data
            
            x_dict['view_mask'][i][1] = 1
        else:
            x_dict['app_list_len'][i] = 1
            x_dict['app_list'][i][1:] = 0
            x_dict['app_list_te_qcut'][i] = 0
            x_dict['view_mask'][i][1] = 0

        if master_id in install_behave_set:
            app_behave = df_install_behave['pkg_id'].at[master_id][-args.app_behave_max_length:]
            len_app = len(app_behave) + 1
            x_dict['app_behave_len'][i] = len_app
            x_dict['app_behave'][i][1: len_app] = app_behave
            x_dict['app_behave'][i][len_app :] = 0
            
#             target_encode_data = np.array(list(df_app_behave_te_qcut.loc[master_id].values))[:, -args.app_behave_max_length:].T
#             x_dict['app_behave_te_qcut'][i][1 : len_app] = target_encode_data
            
            x_dict['app_behave_time_cut'][i][1:len_app] = df_behave_time['cut_id'].at[master_id][-args.app_behave_max_length:]
            x_dict['app_behave_time_qcut'][i][1:len_app] = df_behave_time['qcut_id'].at[master_id][-args.app_behave_max_length:]
            x_dict['app_behave_action'][i][1:len_app] = df_install_behave['action'].at[master_id][-args.app_behave_max_length:]
            x_dict['view_mask'][i][2] = 1
        else:
            x_dict['app_behave_len'][i] = 1
            x_dict['app_behave'][i][1:] = 0
            x_dict['app_behave_te_qcut'][i] = 0

            x_dict['app_behave_time_cut'][i][1:] = 0
            x_dict['app_behave_time_qcut'][i][1:] = 0
            x_dict['app_behave_action'][i][1:] = 0
            x_dict['view_mask'][i][2] = 0
        
        
        if master_id in user_log_set:
            userlog_list = se_userlog_cross.at[master_id][:args.userlog_max_length]
            len_userlog = len(userlog_list) + 1
            x_dict['userlog_len'][i] = len_userlog
            x_dict['userlog'][i][1 : len_userlog] = userlog_list
            x_dict['userlog'][i][len_userlog :] = 0
            x_dict['userlog_day_qcut'][i][1 : len_userlog] = df_userlog_time_seq['qcut_day_id'].at[master_id][:args.userlog_max_length]
            x_dict['userlog_day_qcut'][i][len_userlog :] = 0
            x_dict['userlog_day_cut'][i][1 : len_userlog] = df_userlog_time_seq['cut_day_id'].at[master_id][:args.userlog_max_length]
            x_dict['userlog_day_cut'][i][len_userlog :] = 0
            x_dict['userlog_second_qcut'][i][1 : len_userlog] = df_userlog_time_seq['qcut_second_id'].at[master_id][:args.userlog_max_length]
            x_dict['userlog_second_qcut'][i][len_userlog :] = 0
            x_dict['userlog_second_cut'][i][1 : len_userlog] = df_userlog_time_seq['cut_second_id'].at[master_id][:args.userlog_max_length]
            x_dict['userlog_second_cut'][i][len_userlog :] = 0

            x_dict['view_mask'][i][3] = 1
        else:
            x_dict['userlog_len'][i] = 1
            x_dict['userlog'][i] = 0
            x_dict['userlog_day_qcut'][i] = 0
            x_dict['userlog_day_cut'][i] = 0
            x_dict['userlog_second_qcut'][i] = 0
            x_dict['userlog_second_cut'][i] = 0

            x_dict['view_mask'][i][3] = 0
    
    
    len_id = master_ids.shape[0]
    x_dict['app_list'][len_id:] = 0
    x_dict['app_behave'][len_id:] = 0
    return {
        
        'user_info' : torch.tensor(x_dict['user_info'][:len_id]).float(),
        
        'app_list' : torch.tensor(x_dict['app_list'][:len_id]).long(),
        'app_list_te_qcut' : torch.tensor(x_dict['app_list_te_qcut'][:len_id]).long(),
        'app_list_len' : torch.tensor(x_dict['app_list_len'][:len_id]).long(),
        
        'app_behave' : torch.tensor(x_dict['app_behave'][:len_id]).long(),
        'app_behave_te_qcut' : torch.tensor(x_dict['app_behave_te_qcut'][:len_id]).long(),
        'app_behave_len' : torch.tensor(x_dict['app_behave_len'][:len_id]).long(),
        
        'app_behave_time_cut' : torch.tensor(x_dict['app_behave_time_cut'][:len_id]).long(),
        'app_behave_time_qcut' : torch.tensor(x_dict['app_behave_time_qcut'][:len_id]).long(),
        'app_behave_action' : torch.tensor(x_dict['app_behave_action'][:len_id]).long(),
        'userlog' : torch.tensor(x_dict['userlog'][:len_id]).long(),
        'userlog_len' : torch.tensor(x_dict['userlog_len'][:len_id]).long(),
        'userlog_day_qcut' : torch.tensor(x_dict['userlog_day_qcut'][:len_id]).long(),
        'userlog_day_cut' : torch.tensor(x_dict['userlog_day_cut'][:len_id]).long(),
        'userlog_second_qcut' : torch.tensor(x_dict['userlog_second_qcut'][:len_id]).long(),
        'userlog_second_cut' : torch.tensor(x_dict['userlog_second_cut'][:len_id]).long(),

        'view_mask' : torch.tensor(x_dict['view_mask'][:len_id]).long(),
        'labels1' : torch.tensor(df_target.loc[master_ids]['target_1m30+'].values).long(),
        'labels2' : torch.tensor(df_target.loc[master_ids]['target_2m30+'].values).long(),
        'labels3' : torch.tensor(df_target.loc[master_ids]['target_3m30+'].values).long(),
        'labels4' : torch.tensor(df_target.loc[master_ids]['target_4m30+'].values).long(),
    }

## model

### embed weight

In [14]:
def get_app_list_wv_weight():
    weight = np.zeros((start_app_list_id+1, 100))
    for i in tqdm(range(1, start_app_list_id+1)):
        weight[i] = wv_model_app_list.wv[str(i)]
    weight_tensor = torch.tensor(weight).float()
    return weight_tensor

def get_app_behave_wv_weight():
    weight = np.zeros((start_app_behave_id+1, 100))
    for i in tqdm(range(1, start_app_behave_id+1)):
        weight[i] = wv_model_app_behave.wv[str(i)]
    weight_tensor = torch.tensor(weight).float()
    return weight_tensor

def get_userlog_wv_weight():
    weight = np.zeros((start_uselog_id+1, 100))
    for i in tqdm(range(1, start_uselog_id+1)):
        weight[i] = wv_model_userlog.wv[str(i)]
    weight_tensor = torch.tensor(weight).float()
    return weight_tensor

app_list_weight = get_app_list_wv_weight()
app_behave_weight = get_app_behave_wv_weight()
userlog_weight = get_userlog_wv_weight()

### sub model

In [15]:
def masked_softmax(X, valid_len):
    if valid_len is None:
        return F.softmax(X,dim=-1)
    else:
        shape=X.shape
        if valid_len.dim()==1:
            valid_len=valid_len.view(-1,1).repeat(1,shape[1])
        mask = (torch.arange(0,X.shape[-1]).repeat(X.shape[0],1).to(args.device) < valid_len).repeat(1, X.shape[1]).view(shape)
        
        X = X.masked_fill_(~mask, -float('inf'))
        return F.softmax(X,dim=-1).view(shape)

def make_mask(X, valid_len):
    if valid_len is None:
        return F.softmax(X,dim=-1)
    else:
        shape=X.shape
        if valid_len.dim()==1:
            valid_len=valid_len.view(-1,1).repeat(1,shape[1])

        mask=(torch.arange(0,X.shape[1]).repeat(X.shape[0],1).to(X.device)<valid_len).byte()
        return mask.unsqueeze(2) 

class DotProductAttention(nn.Module):
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)

    # `query`: (`batch_size`, #queries, `d`)
    # `key`: (`batch_size`, #kv_pairs, `d`)
    # `value`: (`batch_size`, #kv_pairs, `dim_v`)
    # `valid_len`: either (`batch_size`, ) or (`batch_size`, xx)
    def forward(self, query, key, value, valid_len=None):
        d = query.shape[-1]
        # Set transpose_b=True to swap the last two dimensions of key
        scores = torch.bmm(query, key.transpose(1,2)) / math.sqrt(d)
        attention_weights = self.dropout(masked_softmax(scores, valid_len))
        return torch.bmm(attention_weights, value)
    
class MLPAttention(nn.Module):
    def __init__(self, key_size, query_size, units, dropout=0., **kwargs):
        super(MLPAttention, self).__init__(**kwargs)
        self.W_k = nn.Linear(key_size, units, bias=False)
        self.W_q = nn.Linear(query_size, units, bias=False)
        self.v = nn.Linear(units, 1, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, valid_len):
        query, key = self.W_k(query), self.W_q(key)
        # Expand query to (`batch_size`, #queries, 1, units), and key to
        # (`batch_size`, 1, #kv_pairs, units). Then plus them with broadcast
        features = query.unsqueeze(2) + key.unsqueeze(1)
        scores = self.v(features).squeeze(-1)
        attention_weights = self.dropout(masked_softmax(scores, valid_len))
        return torch.bmm(attention_weights, value)

class LayerNorm(nn.Module):
    def __init__(self,features,eps=1e-6):
        super(LayerNorm,self).__init__()
        self.gamma=nn.Parameter(torch.ones(features))
        self.beta=nn.Parameter(torch.zeros(features))
        self.eps=eps
    def forward(self,X):
        mean=X.mean(-1,keepdim=True)
        std=X.std(-1,keepdim=True)
        return self.gamma*(X-mean)/(std+self.eps)+self.beta
    
class MLPAttentionPool(nn.Module):
    def __init__(self,key_size,units):
        super().__init__()
        self.proj = nn.Sequential(nn.Linear(key_size,units,bias=False),
                                  nn.Tanh(),
                                  nn.Linear(units,1,bias=False))
        
    def masked_softmax_1d(self, X, valid_len):
        if valid_len is None:
            return F.softmax(X,dim=-1), _
        else:
            shape=X.shape
            if valid_len.dim()==1:
                valid_len=valid_len.view(-1,1).repeat(1,shape[1])

            mask=(torch.arange(0,X.shape[-1]).repeat(X.shape[0],1).to(X.device)<valid_len).byte()
            X = X.masked_fill_(~mask, -float('inf'))
            return F.softmax(X,dim=-1).view(shape), mask

    def forward(self, key, valid_len):
        scores = self.proj(key).squeeze(-1)
        attention_weights, mask = self.masked_softmax_1d(scores,valid_len)
        seq_out = attention_weights.unsqueeze(-1) * key
        return seq_out.sum(1)

class GeLU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1. + torch.tanh(x * 0.7978845608 * (1. + 0.044715 * x * x)))

class Dense(nn.Module):
    def __init__(self, in_feature, out_feature):
        super().__init__()
        hidden = 128
        self.dense = nn.Sequential(
            nn.Linear(in_feature, hidden),
            GeLU(),
            nn.Dropout(args.dropout_rate),
            nn.Linear(hidden, out_feature)
        )
        self.dense.apply(self._init_weights)
        
    def _init_weights(self, module):
        """ Initialize the weights """
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)

    def forward(self, x):
        return self.dense(x)

class UserNetwork(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        n_dim = df_user_one_hot.shape[1]
        self.dense_hidden = Dense(n_dim, config.hidden)

        self.dense1 = Dense(config.hidden, 2)
        self.dense2 = Dense(config.hidden, 2)
        self.dense3 = Dense(config.hidden, 2)
        self.dense4 = Dense(config.hidden, 2)
        
    def forward(self, input_dict):
        
        x = input_dict['user_info'].to(args.device)
        labels1 = input_dict['labels1'].to(args.device)
        labels2 = input_dict['labels2'].to(args.device)
        labels3 = input_dict['labels3'].to(args.device)
        labels4 = input_dict['labels4'].to(args.device)
        
        hidden = self.dense_hidden(x)
        
        y1 = self.dense1(hidden)
        y2 = self.dense2(hidden)
        y3 = self.dense3(hidden)
        y4 = self.dense4(hidden)
        
        loss_func = nn.CrossEntropyLoss()
        loss1 = loss_func(y1, labels1.long())
        loss2 = loss_func(y2, labels2.long())
        loss3 = loss_func(y3, labels3.long())
        loss4 = loss_func(y4, labels4.long())
        
        loss = loss1 + loss2 + loss3 + loss4
        
        return loss, y1, y2, y3, y4, hidden

class AppListNetwork(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.hidden_size = 64
        self.input_size = 100
        self.embeddings = nn.Embedding.from_pretrained(app_list_weight)
        self.layer_norm = LayerNorm(self.input_size)
        
#         self.attention_layer = DotProductAttention(0.)
#         self.attention_layer = MLPAttention(self.input_size, self.input_size, 256)

        self.attention_layer = MLPAttentionPool(self.input_size, config.hidden)
        self.dense_hidden = Dense(self.input_size, config.hidden)
        
        self.dense1 = Dense(config.hidden, 2)
        self.dense2 = Dense(config.hidden, 2)
        self.dense3 = Dense(config.hidden, 2)
        self.dense4 = Dense(config.hidden, 2)
        
    def forward(self, input_dict):
        app_list_ids = input_dict['app_list'].to(args.device)
        app_list_len = input_dict['app_list_len'].to(args.device)
        labels1 = input_dict['labels1'].to(args.device)
        labels2 = input_dict['labels2'].to(args.device)
        labels3 = input_dict['labels3'].to(args.device)
        labels4 = input_dict['labels4'].to(args.device)

        app_list = self.embeddings(app_list_ids)
        app_list = self.layer_norm(app_list)

        x = self.attention_layer(app_list, app_list_len)
        
        hidden = self.dense_hidden(x)        
        y1 = self.dense1(hidden)
        y2 = self.dense2(hidden)
        y3 = self.dense3(hidden)
        y4 = self.dense4(hidden)

        loss_func = nn.CrossEntropyLoss()
        loss1 = loss_func(y1, labels1.long())
        loss2 = loss_func(y2, labels2.long())
        loss3 = loss_func(y3, labels3.long())
        loss4 = loss_func(y4, labels4.long())
        
        loss = loss1 + loss2 + loss3 + loss4
        
        return loss, y1, y2, y3, y4, hidden

class AppBehaveNetwork(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.hidden_size = config.hidden
        self.embeddings = nn.Embedding.from_pretrained(app_behave_weight)        
        for i in self.embeddings.parameters():
            i.requires_grad=False
        
        self.qcut_time_embeddings = nn.Embedding(64, 16)
        self.cut_time_embeddings = nn.Embedding(64, 16)
#         self.action_embeddings = nn.Embedding(2, 4)

        
        self.layer_norm = LayerNorm(100)
        self.rnn = nn.GRU(16 + 16 + 100,
                          hidden_size = config.hidden,
                          num_layers = 1,
                          dropout = 0,
                          bidirectional = False, 
                          batch_first=True)
        
#         self.attention_layer = DotProductAttention(0.)
#         self.attention_layer = MLPAttention(config.hidden, config.hidden, config.hidden)
        self.attention_layer = MLPAttentionPool(config.hidden, config.hidden)

        self.dense_hidden = Dense(config.hidden, config.hidden)

        self.dense1 = Dense(config.hidden, 2)
        self.dense2 = Dense(config.hidden, 2)
        self.dense3 = Dense(config.hidden, 2)
        self.dense4 = Dense(config.hidden, 2)

    def rnn_forward(self, x, x_lens):
        X = torch.nn.utils.rnn.pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        hidden, _= self.rnn(X)
        hidden, _ = torch.nn.utils.rnn.pad_packed_sequence(hidden,total_length=x.shape[1],batch_first=True)
        return hidden
   
    def forward(self, input_dict):
        
        app_behave_ids = input_dict['app_behave'].to(args.device)
        app_behave_len = input_dict['app_behave_len'].to(args.device)
        app_behave_time_cut = input_dict['app_behave_time_cut'].to(args.device)
        app_behave_time_qcut = input_dict['app_behave_time_qcut'].to(args.device)
#         app_behave_action = input_dict['app_behave_action'].to(args.device)
        
        labels1 = input_dict['labels1'].to(args.device)
        labels2 = input_dict['labels2'].to(args.device)
        labels3 = input_dict['labels3'].to(args.device)
        labels4 = input_dict['labels4'].to(args.device)
        
        app_behave = self.embeddings(app_behave_ids)
#         app_behave = self.layer_norm(app_behave)
        cut_time_embed = self.cut_time_embeddings(app_behave_time_cut)
        qcut_time_embed = self.cut_time_embeddings(app_behave_time_qcut)
#         action_embed = self.action_embeddings(app_behave_action)
        
        seq_data = torch.cat([
            app_behave, 
            cut_time_embed,
            qcut_time_embed,
#             action_embed,
        ], dim = -1)
        
        
        rnn_out = self.rnn_forward(seq_data, app_behave_len)
        
        x = self.attention_layer(rnn_out, app_behave_len)
#         mask = make_mask(x, app_behave_len)
#         x = x.masked_fill_(~mask, 0).sum(1)

        
        hidden = self.dense_hidden(x)
        y1 = self.dense1(hidden)
        y2 = self.dense2(hidden)
        y3 = self.dense3(hidden)
        y4 = self.dense4(hidden)
        
        loss_func = nn.CrossEntropyLoss()
        loss1 = loss_func(y1, labels1.long())
        loss2 = loss_func(y2, labels2.long())
        loss3 = loss_func(y3, labels3.long())
        loss4 = loss_func(y4, labels4.long())
        
        loss = loss1 + loss2 + loss3 + loss4
        
        return loss, y1, y2, y3, y4, hidden

class UserlogNetwork(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.hidden_size = config.hidden
        self.embeddings = nn.Embedding.from_pretrained(userlog_weight)        
#         for i in self.embeddings.parameters():
#             i.requires_grad=False
        self.embeddings_day_qcut = nn.Embedding(9, 8)
        self.embeddings_day_cut = nn.Embedding(9, 8)
        self.embeddings_second_qcut = nn.Embedding(33, 16)
        self.embeddings_second_cut = nn.Embedding(33, 16)

        
        self.layer_norm = LayerNorm(100)
        self.rnn = nn.GRU(100 + 8 + 16 + 8 + 16,
                          hidden_size = config.hidden,
                          num_layers = 1,
                          dropout = 0,
                          bidirectional = False, 
                          batch_first=True)
        
#         self.attention_layer = DotProductAttention(0.)
#         self.attention_layer = MLPAttention(config.hidden, config.hidden, config.hidden)
        self.attention_layer = MLPAttentionPool(config.hidden, config.hidden)

        self.dense_hidden = Dense(config.hidden, config.hidden)

        self.dense1 = Dense(config.hidden, 2)
        self.dense2 = Dense(config.hidden, 2)
        self.dense3 = Dense(config.hidden, 2)
        self.dense4 = Dense(config.hidden, 2)

    def rnn_forward(self, x, x_lens):
        X = torch.nn.utils.rnn.pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        hidden, _= self.rnn(X)
        hidden, _ = torch.nn.utils.rnn.pad_packed_sequence(hidden,total_length=x.shape[1],batch_first=True)
        return hidden
   
    def forward(self, input_dict):
        
        userlog_action_id = input_dict['userlog'].to(args.device)
        userlog_len = input_dict['userlog_len'].to(args.device)
        userlog_day_qcut_id = input_dict['userlog_day_qcut'].to(args.device)
        userlog_day_cut_id = input_dict['userlog_day_cut'].to(args.device)
        userlog_second_qcut_id = input_dict['userlog_second_qcut'].to(args.device)
        userlog_second_cut_id = input_dict['userlog_second_cut'].to(args.device)

        
        labels1 = input_dict['labels1'].to(args.device)
        labels2 = input_dict['labels2'].to(args.device)
        labels3 = input_dict['labels3'].to(args.device)
        labels4 = input_dict['labels4'].to(args.device)
        
        userlog_action = self.embeddings(userlog_action_id)
#         app_behave = self.layer_norm(app_behave)
        userlog_day_qcut = self.embeddings_day_qcut(userlog_day_qcut_id)
        userlog_day_cut = self.embeddings_day_cut(userlog_day_cut_id)
        userlog_second_qcut = self.embeddings_second_qcut(userlog_second_qcut_id)
        userlog_second_cut = self.embeddings_second_cut(userlog_second_cut_id)
        hidden = torch.cat([
            userlog_action,
            userlog_day_qcut,
            userlog_day_cut,
            userlog_second_qcut,
            userlog_second_cut], dim = -1)
        
    
        rnn_out = self.rnn_forward(hidden, userlog_len)
        
        x = self.attention_layer(rnn_out, userlog_len)
#         mask = make_mask(x, app_behave_len)
#         x = x.masked_fill_(~mask, 0).sum(1)

        
        hidden = self.dense_hidden(x)
        y1 = self.dense1(hidden)
        y2 = self.dense2(hidden)
        y3 = self.dense3(hidden)
        y4 = self.dense4(hidden)
         
        loss_func = nn.CrossEntropyLoss()
        loss1 = loss_func(y1, labels1.long())
        loss2 = loss_func(y2, labels2.long())
        loss3 = loss_func(y3, labels3.long())
        loss4 = loss_func(y4, labels4.long())
        
        loss = loss1 + loss2 + loss3 + loss4
        
        return loss, y1, y2, y3, y4, hidden

### interactive model

In [16]:
class Alignment(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.temperature = nn.Parameter(torch.tensor(1 / math.sqrt(config.hidden)))
        self.summary = {}

    def _attention(self, a, b):
        return torch.matmul(a, b.transpose(1, 2)) * self.temperature
    
    def forward(self, a, b, mask_a, mask_b):
        attn = self._attention(a, b)
        mask = torch.matmul(mask_a.float(), mask_b.transpose(1, 2).float()).byte()
        attn.masked_fill_(~mask, -1e7)
        attn_a = F.softmax(attn, dim=1)
        attn_b = F.softmax(attn, dim=2)
        feature_b = torch.matmul(attn_a.transpose(1, 2), a)
        feature_a = torch.matmul(attn_b, b)
        return feature_a, feature_b

class MappedAlignment(Alignment):
    def __init__(self, config):
        super().__init__(config)
        self.projection = nn.Sequential(
            nn.Dropout(args.dropout_rate),
            Dense(config.hidden, config.hidden),
        )

    def _attention(self, a, b):
        a = self.projection(a)
        b = self.projection(b)
        return super()._attention(a, b)

class FullFusion(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dropout = args.dropout_rate
        self.fusion1 = Dense(config.hidden * 2, config.hidden)
        self.fusion2 = Dense(config.hidden * 2, config.hidden)
        self.fusion3 = Dense(config.hidden * 2, config.hidden)
        self.fusion = Dense(config.hidden * 3, config.hidden)

    def forward(self, x, align):
        x1 = self.fusion1(torch.cat([x, align], dim=-1))
        x2 = self.fusion2(torch.cat([x, x - align], dim=-1))
        x3 = self.fusion3(torch.cat([x, x * align], dim=-1))
        x = torch.cat([x1, x2, x3], dim=-1)
        x = F.dropout(x, self.dropout, self.training)
        return self.fusion(x)

class AppConcat(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.applist_embeddings = nn.Embedding.from_pretrained(app_list_weight)
        self.appbehave_embeddings = nn.Embedding.from_pretrained(app_behave_weight)
        for i in self.applist_embeddings.parameters():
            i.requires_grad=False
        for i in self.appbehave_embeddings.parameters():
            i.requires_grad=False
        

        self.qcut_time_embeddings = nn.Embedding(64, 16)
        self.cut_time_embeddings = nn.Embedding(64, 16)
        self.action_embeddings = nn.Embedding(2, 4)
        
        self.app_list_qcut_embed_list = nn.ModuleList([nn.Embedding(17, 16) for _ in range(4)])
        self.app_behave_qcut_embed_list = nn.ModuleList([nn.Embedding(17, 16) for _ in range(4)])

        self.app_list_encoder = Dense(100, config.hidden)
#         self.app_list_encoder = Dense(16 * 4, config.hidden)
        self.attention_layer1 = MLPAttentionPool(config.hidden, config.hidden)
        self.attention_layer2 = MLPAttentionPool(config.hidden, config.hidden)


        self.rnn = nn.GRU(16 + 16 + 100,
#         self.rnn = nn.GRU(16 * 4,
                          hidden_size = config.hidden,
                          num_layers = 1,
                          dropout = 0,
                          bidirectional = False, 
                          batch_first=True)        
        
        
        self.dense = Dense(config.hidden * 2, config.hidden)
        self.dense1 = Dense(config.hidden, 2)
        self.dense2 = Dense(config.hidden, 2)
        self.dense3 = Dense(config.hidden, 2)
        self.dense4 = Dense(config.hidden, 2)
        
    def rnn_forward(self, x, x_lens):
        X = torch.nn.utils.rnn.pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        hidden, _= self.rnn(X)
        hidden, _ = torch.nn.utils.rnn.pad_packed_sequence(hidden,total_length=x.shape[1],batch_first=True)
        return hidden
    
    def maxpool(self, x, mask):
        return x.masked_fill_(~mask, -float('inf')).max(dim=1)[0]

    def app_behave_encode(self, input_dict):
        app_behave_ids = input_dict['app_behave'].to(args.device)
        app_behave_len = input_dict['app_behave_len'].to(args.device)
        app_behave_time_cut = input_dict['app_behave_time_cut'].to(args.device)
        app_behave_time_qcut = input_dict['app_behave_time_qcut'].to(args.device)
        app_behave_action = input_dict['app_behave_action'].to(args.device)
   
#         app_behave_target_encode = input_dict['app_behave_target_encode'].to(args.device)
        
        app_behave_target_qcut = input_dict['app_behave_te_qcut'].to(args.device)

#         app_behave_target_qcut_embed1 = self.app_behave_qcut_embed_list[0](app_behave_target_qcut[:, :, 0])
#         app_behave_target_qcut_embed2 = self.app_behave_qcut_embed_list[1](app_behave_target_qcut[:, :, 1])
#         app_behave_target_qcut_embed3 = self.app_behave_qcut_embed_list[2](app_behave_target_qcut[:, :, 2])
#         app_behave_target_qcut_embed4 = self.app_behave_qcut_embed_list[3](app_behave_target_qcut[:, :, 3])

        app_behave = self.appbehave_embeddings(app_behave_ids)
        cut_time_embed = self.cut_time_embeddings(app_behave_time_cut)
        qcut_time_embed = self.cut_time_embeddings(app_behave_time_qcut)
#         action_embed = self.action_embeddings(app_behave_action)
        
        seq_data = torch.cat([
            app_behave, 
            cut_time_embed,
            qcut_time_embed,
#             app_behave_target_qcut_embed1,
#             app_behave_target_qcut_embed2,
#             app_behave_target_qcut_embed3,
#             app_behave_target_qcut_embed4,
#             app_behave_target_encode,
#             action_embed,
        ], dim = -1)
        
        rnn_out = self.rnn_forward(seq_data, app_behave_len)

        return rnn_out, app_behave_len
    
    def app_list_encode(self, input_dict):
        app_list_ids = input_dict['app_list'].to(args.device)
        app_list_len = input_dict['app_list_len'].to(args.device)
        app_list_embed = self.applist_embeddings(app_list_ids)
        
        app_list_target_qcut = input_dict['app_list_te_qcut'].to(args.device)
#         app_list_target_encode = input_dict['app_list_target_encode'].to(args.device)

        app_list_target_qcut_embed1 = self.app_list_qcut_embed_list[0](app_list_target_qcut[:, :, 0])
        app_list_target_qcut_embed2 = self.app_list_qcut_embed_list[1](app_list_target_qcut[:, :, 1])
        app_list_target_qcut_embed3 = self.app_list_qcut_embed_list[2](app_list_target_qcut[:, :, 2])
        app_list_target_qcut_embed4 = self.app_list_qcut_embed_list[3](app_list_target_qcut[:, :, 3])

        seq_data = torch.cat([
            app_list_embed, 
#             app_list_target_encode,
#             app_list_target_qcut_embed1,
#             app_list_target_qcut_embed2,
#             app_list_target_qcut_embed3,
#             app_list_target_qcut_embed4,

        ], dim = -1)
        
        return self.app_list_encoder(seq_data), app_list_len 
    

    def forward(self, input_dict):
        a, a_len = self.app_behave_encode(input_dict)
        b, b_len = self.app_list_encode(input_dict)
#         mask_a = self.make_mask(a, a_len)
#         mask_b = self.make_mask(b, b_len)
                
        a = self.attention_layer1(a, a_len)
        b = self.attention_layer2(b, b_len)
        
        hidden = self.dense(torch.cat([a, b], dim=-1)) 

        labels1 = input_dict['labels1'].to(args.device)
        labels2 = input_dict['labels2'].to(args.device)
        labels3 = input_dict['labels3'].to(args.device)
        labels4 = input_dict['labels4'].to(args.device)
        
        y1 = self.dense1(hidden)
        y2 = self.dense2(hidden)
        y3 = self.dense3(hidden)
        y4 = self.dense4(hidden)

        loss_func = nn.CrossEntropyLoss()
        loss1 = loss_func(y1, labels1.long())
        loss2 = loss_func(y2, labels2.long())
        loss3 = loss_func(y3, labels3.long())
        loss4 = loss_func(y4, labels4.long())
        
        loss = loss1 + loss2 + loss3 + loss4
        
        return loss, y1, y2, y3, y4, hidden
    
class AppInteractiveNetwork(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.applist_embeddings = nn.Embedding.from_pretrained(app_list_weight)
        self.appbehave_embeddings = nn.Embedding.from_pretrained(app_behave_weight)
        for i in self.applist_embeddings.parameters():
            i.requires_grad=False
        for i in self.appbehave_embeddings.parameters():
            i.requires_grad=False
        

        self.qcut_time_embeddings = nn.Embedding(64, 16)
        self.cut_time_embeddings = nn.Embedding(64, 16)
        self.action_embeddings = nn.Embedding(2, 4)
        
        self.app_list_qcut_embed_list = nn.ModuleList([nn.Embedding(17, 16) for _ in range(4)])
        self.app_behave_qcut_embed_list = nn.ModuleList([nn.Embedding(17, 16) for _ in range(4)])

        self.app_list_encoder = Dense(100, config.hidden)
#         self.app_list_encoder = Dense(16 * 4, config.hidden)


        self.rnn = nn.GRU(16 + 16 + 100,
#         self.rnn = nn.GRU(16 * 4,
                          hidden_size = config.hidden,
                          num_layers = 1,
                          dropout = 0,
                          bidirectional = False, 
                          batch_first=True)
        
        self.alignment_layer = MappedAlignment(config)
        self.fusion_layer = FullFusion(config)
        
        
        
        self.dense = Dense(config.hidden * 4, config.hidden)
        self.dense1 = Dense(config.hidden, 2)
        self.dense2 = Dense(config.hidden, 2)
        self.dense3 = Dense(config.hidden, 2)
        self.dense4 = Dense(config.hidden, 2)
        
    def rnn_forward(self, x, x_lens):
        X = torch.nn.utils.rnn.pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        hidden, _= self.rnn(X)
        hidden, _ = torch.nn.utils.rnn.pad_packed_sequence(hidden,total_length=x.shape[1],batch_first=True)
        return hidden
    
    def maxpool(self, x, mask):
        return x.masked_fill_(~mask, -float('inf')).max(dim=1)[0]

    def app_behave_encode(self, input_dict):
        app_behave_ids = input_dict['app_behave'].to(args.device)
        app_behave_len = input_dict['app_behave_len'].to(args.device)
        app_behave_time_cut = input_dict['app_behave_time_cut'].to(args.device)
        app_behave_time_qcut = input_dict['app_behave_time_qcut'].to(args.device)
        app_behave_action = input_dict['app_behave_action'].to(args.device)
   
#         app_behave_target_encode = input_dict['app_behave_target_encode'].to(args.device)
        
#         app_behave_target_qcut = input_dict['app_behave_te_qcut'].to(args.device)

#         app_behave_target_qcut_embed1 = self.app_behave_qcut_embed_list[0](app_behave_target_qcut[:, :, 0])
#         app_behave_target_qcut_embed2 = self.app_behave_qcut_embed_list[1](app_behave_target_qcut[:, :, 1])
#         app_behave_target_qcut_embed3 = self.app_behave_qcut_embed_list[2](app_behave_target_qcut[:, :, 2])
#         app_behave_target_qcut_embed4 = self.app_behave_qcut_embed_list[3](app_behave_target_qcut[:, :, 3])

        app_behave = self.appbehave_embeddings(app_behave_ids)
        cut_time_embed = self.cut_time_embeddings(app_behave_time_cut)
        qcut_time_embed = self.cut_time_embeddings(app_behave_time_qcut)
#         action_embed = self.action_embeddings(app_behave_action)
        
        seq_data = torch.cat([
            app_behave, 
            cut_time_embed,
            qcut_time_embed,
#             app_behave_target_qcut_embed1,
#             app_behave_target_qcut_embed2,
#             app_behave_target_qcut_embed3,
#             app_behave_target_qcut_embed4,
#             app_behave_target_encode,
#             action_embed,
        ], dim = -1)
        
        rnn_out = self.rnn_forward(seq_data, app_behave_len)

        return rnn_out, app_behave_len
    
    def app_list_encode(self, input_dict):
        app_list_ids = input_dict['app_list'].to(args.device)
        app_list_len = input_dict['app_list_len'].to(args.device)
        app_list_embed = self.applist_embeddings(app_list_ids)
        
        app_list_target_qcut = input_dict['app_list_te_qcut'].to(args.device)
#         app_list_target_encode = input_dict['app_list_target_encode'].to(args.device)

        app_list_target_qcut_embed1 = self.app_list_qcut_embed_list[0](app_list_target_qcut[:, :, 0])
        app_list_target_qcut_embed2 = self.app_list_qcut_embed_list[1](app_list_target_qcut[:, :, 1])
        app_list_target_qcut_embed3 = self.app_list_qcut_embed_list[2](app_list_target_qcut[:, :, 2])
        app_list_target_qcut_embed4 = self.app_list_qcut_embed_list[3](app_list_target_qcut[:, :, 3])

        seq_data = torch.cat([
            app_list_embed, 
#             app_list_target_encode,
#             app_list_target_qcut_embed1,
#             app_list_target_qcut_embed2,
#             app_list_target_qcut_embed3,
#             app_list_target_qcut_embed4,

        ], dim = -1)
        
        return self.app_list_encoder(seq_data), app_list_len 
    
    def make_mask(self, X, valid_len):
        shape=X.shape
        if valid_len.dim()==1:
            valid_len=valid_len.view(-1,1).repeat(1,shape[1])
        mask=(torch.arange(0,X.shape[1]).repeat(X.shape[0],1).to(X.device)<valid_len).float()
        return mask.unsqueeze(2).byte()

    def forward(self, input_dict):
        a, a_len = self.app_behave_encode(input_dict)
        b, b_len = self.app_list_encode(input_dict)
        mask_a = self.make_mask(a, a_len)
        mask_b = self.make_mask(b, b_len)
        
        align_a, align_b = self.alignment_layer(a, b, mask_a, mask_b)
        a = self.fusion_layer(a, align_a)
        b = self.fusion_layer(b, align_b)
        
        a = self.maxpool(a, mask_a)
        b = self.maxpool(b, mask_b)
        
        hidden = self.dense(torch.cat([a, b, (a - b).abs(), a * b], dim=-1)) #symmetric


        
        labels1 = input_dict['labels1'].to(args.device)
        labels2 = input_dict['labels2'].to(args.device)
        labels3 = input_dict['labels3'].to(args.device)
        labels4 = input_dict['labels4'].to(args.device)
        
        y1 = self.dense1(hidden)
        y2 = self.dense2(hidden)
        y3 = self.dense3(hidden)
        y4 = self.dense4(hidden)

        loss_func = nn.CrossEntropyLoss()
        loss1 = loss_func(y1, labels1.long())
        loss2 = loss_func(y2, labels2.long())
        loss3 = loss_func(y3, labels3.long())
        loss4 = loss_func(y4, labels4.long())
        
        loss = loss1 + loss2 + loss3 + loss4
        
        return loss, y1, y2, y3, y4, hidden

### multi-view model

In [17]:
class MaskMlpAttention(nn.Module):
    def __init__(self, key_size, units):
        super().__init__()
        self.proj = nn.Sequential(nn.Linear(key_size,units,bias=False),
                                  nn.Tanh(),
                                  nn.Linear(units,1,bias=False))
        
    def forward(self, key, mask):
        scores = self.proj(key).squeeze(-1)
        
        if mask is not None:
            scores = scores.masked_fill_(~mask.byte(), -float('inf'))
            
        softmax_score = F.softmax(scores, dim=-1)
        seq_out = softmax_score.unsqueeze(-1) * key
        return seq_out.sum(1)

class MaskMultiViewEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.user_network = config.user_net
        self.app_list_network = config.app_list_net
        self.app_behave_network = config.app_behave_net
        self.userlog_network = config.userlog_network
        
    def forward(self,input_dict):
        loss_user, y1_user, y2_user, y3_user, y4_user, hidden_user = self.user_network(input_dict)
        loss_list, y1_list, y2_list, y3_list, y4_list, hidden_list = self.app_list_network(input_dict)
        loss_behave, y1_behave, y2_behave, y3_behave, y4_behave, hidden_behave = self.app_behave_network(input_dict)
        loss_userlog, y1_userlog, y2_userlog, y3_userlog, y4_userlog, hidden_userlog = self.userlog_network(input_dict)

        return hidden_user, hidden_list, hidden_behave, hidden_userlog
    
class MaskMultiViewNetwork(nn.Module):
    def __init__(self, config):
        super().__init__()
        
#         self.user_network = UserNetwork(config)
#         self.app_list_network = AppListNetwork(config)
#         self.app_behave_network = AppBehaveNetwork(config)
        
        self.multiview_encoder = MaskMultiViewEncoder(config)

        self.attention = MaskMlpAttention(config.hidden, config.hidden)
#         self.attention = MaskMlpAttentionSeperateMap(config.hidden, config.hidden, 3)

        self.dense = Dense(config.hidden, config.hidden)
        
        self.dense1 = Dense(config.hidden, 2)
        self.dense2 = Dense(config.hidden, 2)
        self.dense3 = Dense(config.hidden, 2)
        self.dense4 = Dense(config.hidden, 2)
        
    def forward(self, input_dict):
        
        hidden_user, hidden_list, hidden_behave, hidden_userlog = self.multiview_encoder(input_dict)
        labels1 = input_dict['labels1'].to(args.device)
        labels2 = input_dict['labels2'].to(args.device)
        labels3 = input_dict['labels3'].to(args.device)
        labels4 = input_dict['labels4'].to(args.device)
        view_mask = input_dict['view_mask'].to(args.device)
        
        multi_view_hidden = torch.cat([hidden_user, hidden_list, hidden_behave, hidden_userlog], dim = 1).view(-1, 4, hidden_user.shape[1])
        hidden = self.attention(multi_view_hidden, view_mask)

        #         hidden = self.attention([hidden_user, hidden_list, hidden_behave], view_mask)
#         hidden = torch.cat([hidden_user, hidden_list, hidden_behave], dim = 1)
        hidden = self.dense(hidden)

        y1 = self.dense1(hidden)
        y2 = self.dense2(hidden)
        y3 = self.dense3(hidden)
        y4 = self.dense4(hidden)
        
        loss_func = nn.CrossEntropyLoss()
        loss1 = loss_func(y1, labels1.long())
        loss2 = loss_func(y2, labels2.long())
        loss3 = loss_func(y3, labels3.long())
        loss4 = loss_func(y4, labels4.long())

        loss = loss1 + loss2 + loss3 + loss4

        return loss, y1, y2, y3, y4, _

class MaskMultiViewNetworkFillZero(nn.Module):
    def __init__(self, config):
        super().__init__()
        
#         self.user_network = UserNetwork(config)
#         self.app_list_network = AppListNetwork(config)
#         self.app_behave_network = AppBehaveNetwork(config)
        
        self.multiview_encoder = MaskMultiViewEncoder(config)

        self.dense = Dense(config.hidden * 4, config.hidden)
        
        self.dense1 = Dense(config.hidden, 2)
        self.dense2 = Dense(config.hidden, 2)
        self.dense3 = Dense(config.hidden, 2)
        self.dense4 = Dense(config.hidden, 2)
        
    def forward(self, input_dict):
        
        hidden_user, hidden_list, hidden_behave, hidden_userlog = self.multiview_encoder(input_dict)
        labels1 = input_dict['labels1'].to(args.device)
        labels2 = input_dict['labels2'].to(args.device)
        labels3 = input_dict['labels3'].to(args.device)
        labels4 = input_dict['labels4'].to(args.device)
        view_mask = input_dict['view_mask'].to(args.device)
        
        hidden_user = view_mask[:, 0:1].float() * hidden_user
        hidden_list = view_mask[:, 1:2].float() * hidden_list
        hidden_behave = view_mask[:, 2:3].float() * hidden_behave
        hidden_userlog = view_mask[:, 3:4].float() * hidden_userlog

        multi_view_hidden = torch.cat([hidden_user, hidden_list, hidden_behave, hidden_userlog], dim = 1)

        #         hidden = self.attention([hidden_user, hidden_list, hidden_behave], view_mask)
#         hidden = torch.cat([hidden_user, hidden_list, hidden_behave], dim = 1)
        hidden = self.dense(multi_view_hidden)

        y1 = self.dense1(hidden)
        y2 = self.dense2(hidden)
        y3 = self.dense3(hidden)
        y4 = self.dense4(hidden)
        
        loss_func = nn.CrossEntropyLoss()
        loss1 = loss_func(y1, labels1.long())
        loss2 = loss_func(y2, labels2.long())
        loss3 = loss_func(y3, labels3.long())
        loss4 = loss_func(y4, labels4.long())

        loss = loss1 + loss2 + loss3 + loss4

        return loss, y1, y2, y3, y4, _

class MaskMultiViewNetworkFillZeroIn(nn.Module):
    def __init__(self, config):
        super().__init__()
        
#         self.user_network = UserNetwork(config)
#         self.app_list_network = AppListNetwork(config)
#         self.app_behave_network = AppBehaveNetwork(config)
        
        self.multiview_encoder = MaskMultiViewEncoderAppInteractive(config)

    
        self.dense = Dense(3 * config.hidden, config.hidden)
        
        self.dense1 = Dense(config.hidden, 2)
        self.dense2 = Dense(config.hidden, 2)
        self.dense3 = Dense(config.hidden, 2)
        self.dense4 = Dense(config.hidden, 2)
        
    def get_3view_mask(self, mask):
        ret = torch.zeros((mask.shape[0], 3))
        ret[:, 0] = mask[:, 0]
        ret[:, 1] = (mask[:, 1] + mask[:, 2]) >= 1
        ret[:, -1] = mask[:, -1]
        return ret

    
    def forward(self, input_dict):
        
        hidden_user, hidden_app, hidden_userlog = self.multiview_encoder(input_dict)
        
        view_mask = self.get_3view_mask(input_dict['view_mask']).to(args.device)
        hidden_user = view_mask[:, 0:1].float() * hidden_user
        hidden_app = view_mask[:, 1:2].float() * hidden_app
        hidden_userlog = view_mask[:, 2:3].float() * hidden_userlog

        labels1 = input_dict['labels1'].to(args.device)
        labels2 = input_dict['labels2'].to(args.device)
        labels3 = input_dict['labels3'].to(args.device)
        labels4 = input_dict['labels4'].to(args.device)
        
        multi_view_hidden = torch.cat([hidden_user, hidden_app, hidden_userlog], dim = 1)

        
        hidden = self.dense(multi_view_hidden)
        
        y1 = self.dense1(hidden)
        y2 = self.dense2(hidden)
        y3 = self.dense3(hidden)
        y4 = self.dense4(hidden)
        
        loss_func = nn.CrossEntropyLoss()
        loss1 = loss_func(y1, labels1.long())
        loss2 = loss_func(y2, labels2.long())
        loss3 = loss_func(y3, labels3.long())
        loss4 = loss_func(y4, labels4.long())

        loss = loss1 + loss2 + loss3 + loss4

        return loss, y1, y2, y3, y4, _

class MaskMultiViewNetworkGenerate(nn.Module):
    def __init__(self, config):
        super().__init__()
        
#         self.user_network = UserNetwork(config)
#         self.app_list_network = AppListNetwork(config)
#         self.app_behave_network = AppBehaveNetwork(config)
        
        self.multiview_encoder = MaskMultiViewEncoder(config)

        self.attention_pool1 = MaskMlpAttention(config.hidden, config.hidden)
#         self.attention = MaskMlpAttentionSeperateMap(config.hidden, config.hidden, 3)
        self.attention_pool2 = MaskMlpAttention(config.hidden, config.hidden)

    
        self.decoder_user_info = Dense(config.hidden, config.hidden)
        self.decoder_app_list = Dense(config.hidden, config.hidden)
        self.decoder_app_behave = Dense(config.hidden, config.hidden)
        self.decoder_user_log = Dense(config.hidden, config.hidden)

        self.dense = Dense(config.hidden, config.hidden)
        
        self.dense1 = Dense(config.hidden, 2)
        self.dense2 = Dense(config.hidden, 2)
        self.dense3 = Dense(config.hidden, 2)
        self.dense4 = Dense(config.hidden, 2)
        
    def forward(self, input_dict):
        
        hidden_user, hidden_list, hidden_behave, hidden_userlog = self.multiview_encoder(input_dict)
        labels1 = input_dict['labels1'].to(args.device)
        labels2 = input_dict['labels2'].to(args.device)
        labels3 = input_dict['labels3'].to(args.device)
        labels4 = input_dict['labels4'].to(args.device)
        view_mask = input_dict['view_mask'].to(args.device)
        
        multi_view_hidden = torch.stack([hidden_user, hidden_list, hidden_behave, hidden_userlog], dim = 1)

        hidden = self.attention_pool1(multi_view_hidden, view_mask)

        #         hidden = self.attention([hidden_user, hidden_list, hidden_behave], view_mask)
#         hidden = torch.cat([hidden_user, hidden_list, hidden_behave], dim = 1)

        generate_user = self.decoder_user_info(hidden)
        generate_list = self.decoder_app_list(hidden)
        generate_behave = self.decoder_app_behave(hidden)
        generate_userlog = self.decoder_user_log(hidden)
        
        
        loss_rebuild_user = (hidden_user - generate_user) ** 2 * view_mask[:, 0:1].float()
        loss_rebuild_list = (hidden_list - generate_list) ** 2 * view_mask[:, 1:2].float()
        loss_rebuild_behave = (hidden_behave - generate_behave) ** 2 * view_mask[:, 2:3].float()
        loss_rebuild_userlog = (hidden_userlog - generate_userlog) ** 2 * view_mask[:, 3:4].float()
        loss_rebuild = loss_rebuild_user.mean() + loss_rebuild_list.mean() + loss_rebuild_behave.mean() + loss_rebuild_userlog.mean()
        
        multi_view_hidden_generate = torch.stack([generate_user, generate_list, generate_behave, generate_userlog], dim = 1)
        hidden_generate = self.attention_pool2(multi_view_hidden_generate, None)
        hidden = self.dense(hidden_generate)

        y1 = self.dense1(hidden)
        y2 = self.dense2(hidden)
        y3 = self.dense3(hidden)
        y4 = self.dense4(hidden)
        
        loss_func = nn.CrossEntropyLoss()
        loss1 = loss_func(y1, labels1.long())
        loss2 = loss_func(y2, labels2.long())
        loss3 = loss_func(y3, labels3.long())
        loss4 = loss_func(y4, labels4.long())

        loss = loss1 + loss2 + loss3 + loss4 + loss_rebuild

        return loss, y1, y2, y3, y4, _

class MaskMultiViewNetworkGenerateWithResidual(nn.Module):
    def __init__(self, config):
        super().__init__()
        
#         self.user_network = UserNetwork(config)
#         self.app_list_network = AppListNetwork(config)
#         self.app_behave_network = AppBehaveNetwork(config)
        
        self.multiview_encoder = MaskMultiViewEncoder(config)

        self.attention_pool1 = MaskMlpAttention(config.hidden, config.hidden)
#         self.attention = MaskMlpAttentionSeperateMap(config.hidden, config.hidden, 3)
        self.attention_pool2 = MaskMlpAttention(config.hidden, config.hidden)

    
        self.decoder_user_info = Dense(config.hidden, config.hidden)
        self.decoder_app_list = Dense(config.hidden, config.hidden)
        self.decoder_app_behave = Dense(config.hidden, config.hidden)
        self.decoder_user_log = Dense(config.hidden, config.hidden)

        self.dense = Dense(config.hidden, config.hidden)
        
        self.dense1 = Dense(config.hidden, 2)
        self.dense2 = Dense(config.hidden, 2)
        self.dense3 = Dense(config.hidden, 2)
        self.dense4 = Dense(config.hidden, 2)
                
    def forward(self, input_dict):
        
        hidden_user, hidden_list, hidden_behave, hidden_userlog = self.multiview_encoder(input_dict)
        labels1 = input_dict['labels1'].to(args.device)
        labels2 = input_dict['labels2'].to(args.device)
        labels3 = input_dict['labels3'].to(args.device)
        labels4 = input_dict['labels4'].to(args.device)
        view_mask = input_dict['view_mask'].to(args.device)
        
        multi_view_hidden = torch.stack([hidden_user, hidden_list, hidden_behave, hidden_userlog], dim = 1)

        hidden = self.attention_pool1(multi_view_hidden, view_mask)

        #         hidden = self.attention([hidden_user, hidden_list, hidden_behave], view_mask)
#         hidden = torch.cat([hidden_user, hidden_list, hidden_behave], dim = 1)

        generate_user = self.decoder_user_info(hidden)
        generate_list = self.decoder_app_list(hidden)
        generate_behave = self.decoder_app_behave(hidden)
        generate_userlog = self.decoder_user_log(hidden)
        
        
        loss_rebuild_user = (hidden_user - generate_user) ** 2 * view_mask[:, 0:1].float()
        loss_rebuild_list = (hidden_list - generate_list) ** 2 * view_mask[:, 1:2].float()
        loss_rebuild_behave = (hidden_behave - generate_behave) ** 2 * view_mask[:, 2:3].float()
        loss_rebuild_userlog = (hidden_userlog - generate_userlog) ** 2 * view_mask[:, 3:4].float()
        loss_rebuild = loss_rebuild_user.mean() + loss_rebuild_list.mean() + loss_rebuild_behave.mean() + loss_rebuild_userlog.mean()
        
        multi_view_hidden_generate = torch.stack([generate_user, generate_list, generate_behave, generate_userlog], dim = 1)
        hidden_generate = self.attention_pool2(multi_view_hidden_generate, None)
        
        hidden = hidden + hidden_generate
        hidden = self.dense(hidden)
        
        y1 = self.dense1(hidden)
        y2 = self.dense2(hidden)
        y3 = self.dense3(hidden)
        y4 = self.dense4(hidden)
        
        loss_func = nn.CrossEntropyLoss()
        loss1 = loss_func(y1, labels1.long())
        loss2 = loss_func(y2, labels2.long())
        loss3 = loss_func(y3, labels3.long())
        loss4 = loss_func(y4, labels4.long())

        loss = loss1 + loss2 + loss3 + loss4 + loss_rebuild

        return loss, y1, y2, y3, y4, _

class MaskMultiViewEncoderAppInteractive(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.user_network = config.user_net
        self.app_network = config.app_net
        self.userlog_network = config.userlog_network
        
    def forward(self,input_dict):
        loss_user, y1_user, y2_user, y3_user, y4_user, hidden_user = self.user_network(input_dict)
        loss_app, y1_app, y2_app, y3_app, y4_app, hidden_app = self.app_network(input_dict)
        loss_userlog, y1_userlog, y2_userlog, y3_userlog, y4_userlog, hidden_userlog = self.userlog_network(input_dict)

        return hidden_user, hidden_app, hidden_userlog

class MaskMultiViewNetworkGenerateWithResidualAppInteractive(nn.Module):
    def __init__(self, config):
        super().__init__()
        
#         self.user_network = UserNetwork(config)
#         self.app_list_network = AppListNetwork(config)
#         self.app_behave_network = AppBehaveNetwork(config)
        
        self.multiview_encoder = MaskMultiViewEncoderAppInteractive(config)

        self.attention_pool1 = MaskMlpAttention(config.hidden, config.hidden)
#         self.attention = MaskMlpAttentionSeperateMap(config.hidden, config.hidden, 3)
        self.attention_pool2 = MaskMlpAttention(config.hidden, config.hidden)
    
        self.decoder_user_info = Dense(config.hidden, config.hidden)
        self.decoder_app = Dense(config.hidden, config.hidden)
        self.decoder_user_log = Dense(config.hidden, config.hidden)

        self.dense = Dense(config.hidden, config.hidden)
        
        self.dense1 = Dense(config.hidden, 2)
        self.dense2 = Dense(config.hidden, 2)
        self.dense3 = Dense(config.hidden, 2)
        self.dense4 = Dense(config.hidden, 2)
    
    def get_3view_mask(self, mask):
        ret = torch.zeros((mask.shape[0], 3))
        ret[:, 0] = mask[:, 0]
        ret[:, 1] = (mask[:, 1] + mask[:, 2]) >= 1
        ret[:, -1] = mask[:, -1]
        return ret
    
    def forward(self, input_dict):
        
        hidden_user, hidden_app, hidden_userlog = self.multiview_encoder(input_dict)
        labels1 = input_dict['labels1'].to(args.device)
        labels2 = input_dict['labels2'].to(args.device)
        labels3 = input_dict['labels3'].to(args.device)
        labels4 = input_dict['labels4'].to(args.device)
        view_mask = self.get_3view_mask(input_dict['view_mask']).to(args.device)
        
        multi_view_hidden = torch.stack([hidden_user, hidden_app, hidden_userlog], dim = 1)

        hidden = self.attention_pool1(multi_view_hidden, view_mask)

        #         hidden = self.attention([hidden_user, hidden_list, hidden_behave], view_mask)
#         hidden = torch.cat([hidden_user, hidden_list, hidden_behave], dim = 1)

        generate_user = self.decoder_user_info(hidden)
        generate_app = self.decoder_app(hidden)
        generate_userlog = self.decoder_user_log(hidden)
        
        
        loss_rebuild_user = (hidden_user - generate_user) ** 2 * view_mask[:, 0:1].float()
        loss_rebuild_app = (hidden_app - generate_app) ** 2 * view_mask[:, 1:2].float()
        loss_rebuild_userlog = (hidden_userlog - generate_userlog) ** 2 * view_mask[:, 2:3].float()
        loss_rebuild = loss_rebuild_user.mean() + loss_rebuild_app.mean()  + loss_rebuild_userlog.mean()
        
        multi_view_hidden_generate = torch.stack([generate_user, generate_app, generate_userlog], dim = 1)
        hidden_generate = self.attention_pool2(multi_view_hidden_generate, None)
        
        hidden = hidden + hidden_generate
        hidden = self.dense(hidden)

        y1 = self.dense1(hidden)
        y2 = self.dense2(hidden)
        y3 = self.dense3(hidden)
        y4 = self.dense4(hidden)
        
        loss_func = nn.CrossEntropyLoss()
        loss1 = loss_func(y1, labels1.long())
        loss2 = loss_func(y2, labels2.long())
        loss3 = loss_func(y3, labels3.long())
        loss4 = loss_func(y4, labels4.long())

        loss = loss1 + loss2 + loss3 + loss4 + loss_rebuild

        return loss, y1, y2, y3, y4, hidden


## trainer

In [18]:
from sklearn.metrics import f1_score

def best_f1(pre_score, label):
        
    max_score = 0
    r = 0.
    while r < 0.5:
        r += 0.005
        y_pre = pre_score > r
        score = f1_score(y_pre, label)
        max_score = max(max_score, score)
    return max_score

best_f1(np.array([0.02,0.2,0.01]), np.array([0,1,1]))

0.8

In [19]:
data = np.array(
[
    [1,2],
    [3,99]
]
)
softmax(data, 1)

array([[2.68941421e-01, 7.31058579e-01],
       [2.03109266e-42, 1.00000000e+00]])

In [25]:
from sklearn.metrics import f1_score


def eval_data(model, master_ids):
    
    if(len(master_ids) > args.n_eval):
        master_ids = random.sample(master_ids, args.n_eval)

    torch_dataset = AppDataset(master_ids)
    data_loader = Data.DataLoader(
        dataset=torch_dataset,      
        batch_size=args.batch_size,      
        shuffle=False,
        collate_fn=collate_fn,
        num_workers = args.n_worker,
    )
    
    loss_list = []
    y1_list, y2_list, y3_list, y4_list = [], [], [], []
    label1_list, label2_list, label3_list, label4_list = [], [], [], []
#     for step, data in enumerate(tqdm(data_loader)):
    with torch.no_grad():
        for step, data in enumerate(tqdm(data_loader)):
            loss, y1, y2, y3, y4, _ = model(data)

            loss_list.append(loss.item())
            y1_list.append(y1.cpu().detach().numpy())
            y2_list.append(y2.cpu().detach().numpy())
            y3_list.append(y3.cpu().detach().numpy())
            y4_list.append(y4.cpu().detach().numpy())

            label1_list.append(data['labels1'].cpu().detach().numpy())
            label2_list.append(data['labels2'].cpu().detach().numpy())
            label3_list.append(data['labels3'].cpu().detach().numpy())
            label4_list.append(data['labels4'].cpu().detach().numpy())

        loss = np.mean(loss_list)
    
    y1_np = np.concatenate(y1_list,  axis = 0)
    y2_np = np.concatenate(y2_list,  axis = 0)
    y3_np = np.concatenate(y3_list,  axis = 0)
    y4_np = np.concatenate(y4_list,  axis = 0)

#     y1_np = softmax(y1_np, 1)
#     y2_np = softmax(y2_np, 1)
#     y3_np = softmax(y3_np, 1)
#     y4_np = softmax(y4_np, 1)

    labels1_np = np.concatenate(label1_list,  axis = 0)
    labels2_np = np.concatenate(label2_list,  axis = 0)
    labels3_np = np.concatenate(label3_list,  axis = 0)
    labels4_np = np.concatenate(label4_list,  axis = 0)

    auc1 = roc_auc_score(labels1_np, y1_np[:, 1])
    auc2 = roc_auc_score(labels2_np, y2_np[:, 1])
    auc3 = roc_auc_score(labels3_np, y3_np[:, 1])
    auc4 = roc_auc_score(labels4_np, y4_np[:, 1])
    auc_all = [auc1, auc2, auc3, auc4]
    
    new_client = np.array(df_master_records['new_client'].loc[master_ids].values)
    new_client_auc1 = roc_auc_score(labels1_np[new_client], y1_np[:, 1][new_client])
    new_client_auc2 = roc_auc_score(labels2_np[new_client], y2_np[:, 1][new_client])
    new_client_auc3 = roc_auc_score(labels3_np[new_client], y3_np[:, 1][new_client])
    new_client_auc4 = roc_auc_score(labels4_np[new_client], y4_np[:, 1][new_client])
    auc_new_client = [new_client_auc1, new_client_auc2, new_client_auc3, new_client_auc4]

    old_client = ~new_client
    old_client_auc1 = roc_auc_score(labels1_np[old_client], y1_np[:, 1][old_client])
    old_client_auc2 = roc_auc_score(labels2_np[old_client], y2_np[:, 1][old_client])
    old_client_auc3 = roc_auc_score(labels3_np[old_client], y3_np[:, 1][old_client])
    old_client_auc4 = roc_auc_score(labels4_np[old_client], y4_np[:, 1][old_client])
    auc_old_client = [old_client_auc1, old_client_auc2, old_client_auc3, old_client_auc4]

    f11 = best_f1(labels1_np, y1_np[:, 1])
    f12 = best_f1(labels2_np, y2_np[:, 1])
    f13 = best_f1(labels3_np, y3_np[:, 1])
    f14 = best_f1(labels4_np, y4_np[:, 1])
    f1_all = [f11, f12, f13, f14]
    
    new_client = np.array(df_master_records['new_client'].loc[master_ids].values)
    new_client_f1 = best_f1(labels1_np[new_client], y1_np[:, 1][new_client])
    new_client_f2 = best_f1(labels2_np[new_client], y2_np[:, 1][new_client])
    new_client_f3 = best_f1(labels3_np[new_client], y3_np[:, 1][new_client])
    new_client_f4 = best_f1(labels4_np[new_client], y4_np[:, 1][new_client])
    auc_new_client = [new_client_f1, new_client_f2, new_client_f3, new_client_f4]

    old_client = ~new_client
    old_client_f1 = best_f1(labels1_np[old_client], y1_np[:, 1][old_client])
    old_client_f2 = best_f1(labels2_np[old_client], y2_np[:, 1][old_client])
    old_client_f3 = best_f1(labels3_np[old_client], y3_np[:, 1][old_client])
    old_client_f4 = best_f1(labels4_np[old_client], y4_np[:, 1][old_client])
    auc_old_client = [old_client_f1, old_client_f2, old_client_f3, old_client_f4]

    
    return {
        'loss' : loss,
        '1m30+' : auc1,
        '2m30+' : auc2,
        '3m30+' : auc3,
        '4m30+' : auc4,
        'new_1m30+' : new_client_auc1,
        'new_2m30+' : new_client_auc2,
        'new_3m30+' : new_client_auc3,
        'new_4m30+' : new_client_auc4,
        'old_1m30+' : old_client_auc1,
        'old_2m30+' : old_client_auc2,
        'old_3m30+' : old_client_auc3,
        'old_4m30+' : old_client_auc4
    },{
        'f1_1m30+' : f11,
        'f1_2m30+' : f12,
        'f1_3m30+' : f13,
        'f1_4m30+' : f14,
        'f1_new_1m30+' : new_client_f1,
        'f1_new_2m30+' : new_client_f2,
        'f1_new_3m30+' : new_client_f3,
        'f1_new_4m30+' : new_client_f4,
        'f1_old_1m30+' : old_client_f1,
        'f1_old_2m30+' : old_client_f2,
        'f1_old_3m30+' : old_client_f3,
        'f1_old_4m30+' : old_client_f4,

    }

def train(train_ids, test_ids, model_class, config):
        
    torch_dataset = AppDataset(train_ids)
    data_loader = Data.DataLoader(
        dataset=torch_dataset,      
        batch_size=args.batch_size,      
        shuffle=True,
        collate_fn=collate_fn,
        num_workers = args.n_worker,
    )
    
    model = model_class(config).to(args.device)
#     model = UserNetwork().to(args.device)
#     model = AppBehaveNetwork().to(args.device)
#     model = AppListNetwork().to(args.device)
#     model = MaskMultiViewNetwork(config).to(args.device)
#     optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.001)

    decay = ["app_network"]

    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in decay)], "weight_decay": args.weight_decay},
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr = args.lr, weight_decay = args.weight_decay)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=int(len(train_ids)//(args.batch_size)), num_training_steps=int(len(train_ids) / args.batch_size * args.epoch)
    )

    for epoch in range(args.epoch):
        model.train()
        for step, data in enumerate(tqdm(data_loader)):
            loss, y1, y2, y3, y4, _ = model(data)            
            #backward
            S = time.time()
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 2)
            optimizer.step()
            scheduler.step()
        
        model.eval()
        train_ret_dict = eval_data(model, train_ids)
        test_ret_dict = eval_data(model, test_ids)
        df_ret = pd.DataFrame([
            train_ret_dict, test_ret_dict
        ], index = ['train', 'test'])
        
        
#         install_behave_test = list(set(install_behave_set) & set(test_ids))
#         install_list_test = list(set(install_list_set) & set(test_ids))
#         user_info_test = list(set(user_info_set) & set(test_ids))
#         user_log_test = list(set(user_log_set) & set(test_ids))
        
#         behave_ret_dict = eval_data(model, install_behave_test)
#         list_ret_dict = eval_data(model, install_list_test)
#         userinfo_ret_dict = eval_data(model, user_info_test)
#         userlog_ret_dict = eval_data(model, user_log_test)

#         df_ret = pd.DataFrame([
#             test_ret_dict, behave_ret_dict, list_ret_dict, userinfo_ret_dict, userlog_ret_dict
#         ], index = ['test', 'behave', 'list', 'user_info', 'user_log'])
        

        logging.info('epoch : %d' % epoch)
        ipd.display(df_ret)
#         torch.save(model, 'model/multiview_net_extend.torch.model_%d' % epoch)

        
    return model

## config

In [26]:
class UserNetConfig():
    _instance = None
    def __new__(cls, *args, **kw):
        if cls._instance is None:
            cls._instance = object.__new__(cls, *args, **kw)
        return cls._instance
    
    def __init__(self):
        super().__init__()
        self.hidden = 256

class AppbehaveConfig():
    _instance = None
    def __new__(cls, *args, **kw):
        if cls._instance is None:
            cls._instance = object.__new__(cls, *args, **kw)
        return cls._instance
    
    def __init__(self):
        super().__init__()
        self.hidden = 256
        
class AppListConfig():
    _instance = None
    def __new__(cls, *args, **kw):
        if cls._instance is None:
            cls._instance = object.__new__(cls, *args, **kw)
        return cls._instance
    
    def __init__(self):
        super().__init__()
        self.hidden = 256
        
class Userlogconfig():
    _instance = None
    def __new__(cls, *args, **kw):
        if cls._instance is None:
            cls._instance = object.__new__(cls, *args, **kw)
        return cls._instance
    
    def __init__(self):
        super().__init__()
        self.hidden = 256

class MultiviewConfig():
    _instance = None
    def __new__(cls, *args, **kw):
        if cls._instance is None:
            cls._instance = object.__new__(cls, *args, **kw)
        return cls._instance
    
    def __init__(self):
        super().__init__()
        self.hidden = 256
#         self.app_list_net = torch.load('app_list_net.model.torch')
#         self.app_behave_net =  torch.load('app_list_net.model.torch')
#         self.user_net =  torch.load('user_net.model.torch')
        self.app_list_net = AppListNetwork(AppListConfig())
        self.app_behave_net = AppBehaveNetwork(AppbehaveConfig())
        self.user_net = UserNetwork(UserNetConfig())
        self.userlog_network = UserlogNetwork(Userlogconfig())

class MultiviewAppInteractiveConfig():
    _instance = None
    def __new__(cls, *args, **kw):
        if cls._instance is None:
            cls._instance = object.__new__(cls, *args, **kw)
        return cls._instance
    
    def __init__(self):
        super().__init__()
        self.hidden = 256
#         self.app_list_net = torch.load('app_list_net.model.torch')
#         self.app_behave_net =  torch.load('app_list_net.model.torch')
#         self.user_net =  torch.load('user_net.model.torch')
        self.app_net = AppInteractiveNetwork(AppListConfig())
        self.user_net = UserNetwork(UserNetConfig())
        self.userlog_network = UserlogNetwork(Userlogconfig())

## model-train

### user attribute

In [27]:
logging.info('start training')
        
user_net_config = UserNetConfig()

user_net = train(all_train_id, all_test_id, UserNetwork, user_net_config)
torch.save(user_net, 'user_net.model.torch')
logging.info('finish training')

2020-11-21 13:55:24,397 - INFO - start training


2020-11-21 13:59:29,847 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.165257,0.613872,0.616249,0.606342,0.600528,0.612236,0.621997,0.608967,0.601824,0.615787,0.613441,0.598940,0.596565
test,1.126142,0.590021,0.585952,0.577175,0.579137,0.596796,0.596786,0.584595,0.585138,0.612384,0.587241,0.576371,0.581438


2020-11-21 14:03:31,277 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.162432,0.619185,0.630995,0.616834,0.612715,0.614436,0.623428,0.611888,0.604630,0.601946,0.635001,0.614463,0.612099
test,1.121832,0.600961,0.603718,0.590773,0.595469,0.606408,0.610460,0.598481,0.600618,0.627813,0.608520,0.594987,0.602514


...

2020-11-21 14:39:59,679 - INFO - epoch : 10


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.161137,0.645410,0.638311,0.626027,0.623461,0.636926,0.638079,0.625636,0.618313,0.633678,0.636030,0.618042,0.618888
test,1.119509,0.597985,0.602078,0.592812,0.600625,0.597092,0.610648,0.598555,0.602957,0.619332,0.607447,0.596334,0.605636


2020-11-21 14:44:01,689 - INFO - epoch : 11


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.150786,0.644050,0.645126,0.629366,0.627875,0.635220,0.641582,0.625531,0.620396,0.645283,0.651309,0.628009,0.626945
test,1.117964,0.597352,0.602193,0.592697,0.601105,0.596671,0.610631,0.597366,0.602549,0.617084,0.607778,0.596702,0.606361


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UserNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Dense. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GeLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
2020-11-21 14:44:01,714 - INFO - finish training


In [22]:
user_net = torch.load('user_net.model.torch')

In [30]:
pd.Series([1,2,3,3,2]).value_counts(1)[1]

0.2

In [31]:
def best_f1(label, pre_score):
    
    ipd.display(pd.DataFrame(np.array(pre_score).reshape(-1, 1)).describe())
    max_score = 0
    select_score = random.sample(list(pre_score), 10000)

    
    defalt_rate = pd.Series(label).value_counts(1)[0]
    r = sorted(pre_score)[int(len(pre_score)*defalt_rate)]
    y_pre = pre_score > r
    score = f1_score(y_pre, label, average='micro')
    print(score)
    
    return score

auc_ret_dict, f1_ret_dict = eval_data(user_net, all_test_id)
df_ret = pd.DataFrame([
    f1_ret_dict
], index = ['test'])
ipd.display(df_ret)

,0
count,75896.000000
mean,-1.528060
std,0.227755
min,-2.616210
25%,-1.686581
50%,-1.527498
75%,-1.369094
max,0.520492


0.9443712448613892


,0
count,75896.000000
mean,-1.092095
std,0.296842
min,-3.220328
25%,-1.298535
50%,-1.071883
75%,-0.869126
max,0.447048


0.8537077052809108


,0
count,75896.000000
mean,-1.187799
std,0.236532
min,-3.122990
25%,-1.331252
50%,-1.183380
75%,-1.030346
max,0.456307


...

,0
count,51524.000000
mean,-1.188511
std,0.236876
min,-3.010246
25%,-1.331437
50%,-1.183305
75%,-1.029564
max,0.456307


0.890128871981989


,0
count,51524.000000
mean,-1.225049
std,0.342680
min,-3.464868
25%,-1.447023
50%,-1.172386
75%,-0.944862
max,0.335686


0.8665670367207515


,f1_1m30+,f1_2m30+,f1_3m30+,f1_4m30+,f1_new_1m30+,f1_new_2m30+,f1_new_3m30+,f1_new_4m30+,f1_old_1m30+,f1_old_2m30+,f1_old_3m30+,f1_old_4m30+
test,0.944371,0.853708,0.823153,0.798408,0.866773,0.748359,0.714057,0.69227,0.98523,0.917534,0.890129,0.866567


### App behave

In [28]:
app_behave_ids = list(df_install_behave.index)
train_app_behave_ids = list( set(app_behave_ids) & set(all_train_id) )
test_app_behave_ids = list( set(app_behave_ids) & set(all_test_id) )
len(train_app_behave_ids), len(test_app_behave_ids)

(154835, 42589)

In [29]:
logging.info('start training')

app_behave_net = train(train_app_behave_ids, test_app_behave_ids, AppBehaveNetwork, AppbehaveConfig())
torch.save(app_behave_net, 'app_behave_net.model.torch')
logging.info('finish training')


2020-11-21 14:44:01,869 - INFO - start training


2020-11-21 14:47:53,337 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.258172,0.64309,0.642131,0.631966,0.628085,0.601041,0.601075,0.595464,0.594055,0.635549,0.640923,0.627183,0.618700
test,1.155376,0.60065,0.605210,0.596819,0.591103,0.563864,0.567669,0.563159,0.561850,0.589873,0.604080,0.593257,0.582926


2020-11-21 14:51:46,230 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.219615,0.683323,0.676511,0.667398,0.662446,0.634015,0.636386,0.628219,0.629111,0.683689,0.675467,0.663639,0.652396
test,1.148613,0.627181,0.616978,0.607517,0.599715,0.575528,0.574556,0.565952,0.563790,0.634622,0.617862,0.606567,0.593785


...

2020-11-21 15:26:59,839 - INFO - epoch : 10


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.364780,0.975022,0.983023,0.983769,0.967661,0.959122,0.976278,0.977506,0.961348,0.985838,0.986132,0.987388,0.967853
test,1.914963,0.664787,0.649398,0.634631,0.618073,0.628382,0.612613,0.600101,0.589028,0.701275,0.667887,0.646826,0.620835


2020-11-21 15:30:50,795 - INFO - epoch : 11


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.341246,0.977675,0.984833,0.985464,0.969713,0.963848,0.978729,0.980156,0.963353,0.987119,0.988288,0.988581,0.971453
test,2.084938,0.663561,0.647403,0.633227,0.616349,0.627977,0.611489,0.599344,0.588009,0.699069,0.665907,0.645501,0.619114


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type AppBehaveNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LayerNorm. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MLPAttentionPool. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Dense. It won't

### App list

In [30]:
app_list_ids = se_id_install_list.index
train_app_list_ids = list( set(app_list_ids) & set(all_train_id) )
test_app_list_ids = list( set(app_list_ids) & set(all_test_id) )
len(train_app_list_ids), len(test_app_list_ids)

(139440, 38950)

In [31]:
logging.info('start training')
app_list_net = train(train_app_list_ids, test_app_list_ids, AppListNetwork, AppListConfig())
torch.save(app_list_net, 'app_list_net.model.torch')
logging.info('finish training')

2020-11-21 15:30:51,746 - INFO - start training


2020-11-21 15:33:23,798 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.235634,0.668588,0.668101,0.661916,0.651864,0.616375,0.624593,0.622779,0.61719,0.666160,0.670364,0.654511,0.639355
test,1.120506,0.650660,0.642583,0.629551,0.620789,0.606309,0.612531,0.596324,0.58727,0.653778,0.633962,0.621851,0.612797


2020-11-21 15:35:56,611 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.223831,0.683952,0.673673,0.668080,0.659906,0.628467,0.628237,0.626844,0.624747,0.663594,0.667802,0.657623,0.643017
test,1.126327,0.656567,0.647334,0.632386,0.622779,0.600518,0.608138,0.595338,0.587145,0.664322,0.636223,0.619852,0.609893


...

2020-11-21 15:58:42,501 - INFO - epoch : 10


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.182668,0.719218,0.707677,0.703811,0.692282,0.664387,0.663837,0.662181,0.653830,0.727354,0.712265,0.701133,0.686811
test,1.118642,0.660752,0.647353,0.635320,0.626956,0.615474,0.618748,0.608178,0.599929,0.677217,0.640401,0.620315,0.612643


2020-11-21 16:01:14,554 - INFO - epoch : 11


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.179101,0.716885,0.711446,0.707376,0.696499,0.662213,0.665061,0.663939,0.658217,0.720961,0.718884,0.707032,0.690216
test,1.119576,0.660581,0.645453,0.634603,0.626973,0.614059,0.616497,0.606873,0.599231,0.679173,0.637748,0.619475,0.612808


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type AppListNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LayerNorm. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MLPAttentionPool. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Dense. It won't b

### App interation

In [17]:
app_ids = set(se_id_install_list.index) | set(df_install_behave.index)
train_app_ids = list( set(app_ids) & set(all_train_id) )
test_app_ids = list( set(app_ids) & set(all_test_id) )
len(train_app_ids), len(test_app_ids)

(171635, 46887)

#### origin

In [27]:
logging.info('start training')
app_interactive_net = train(train_app_ids, test_app_ids, AppInteractiveNetwork, AppListConfig())
torch.save(app_interactive_net, 'app_interactive_net.model.torch')
logging.info('finish training')

2020-09-04 14:35:39,045 - INFO - start training


2020-09-04 14:45:05,496 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.095028,0.680940,0.791862,0.789788,0.785082,0.652764,0.728408,0.718859,0.717533,0.716179,0.834416,0.832668,0.819506
test,1.102913,0.615572,0.670529,0.662052,0.646112,0.593985,0.631558,0.620439,0.611380,0.649865,0.673745,0.664858,0.641043


2020-09-04 14:54:29,675 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.10595,0.723106,0.795601,0.799308,0.791596,0.693909,0.739168,0.738522,0.730635,0.764185,0.839362,0.841182,0.827283
test,1.11165,0.617647,0.669183,0.664551,0.649665,0.595341,0.630261,0.625920,0.616004,0.646972,0.676009,0.669246,0.647619


...

2020-09-04 16:19:12,274 - INFO - epoch : 10


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.554112,0.954974,0.958231,0.958856,0.945621,0.935360,0.944568,0.944676,0.929592,0.967587,0.965531,0.966890,0.951666
test,1.449294,0.656798,0.663796,0.657603,0.646252,0.620111,0.628644,0.622896,0.615542,0.691190,0.675820,0.667587,0.648112


2020-09-04 16:28:38,802 - INFO - epoch : 11


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.525389,0.958865,0.962740,0.962992,0.948635,0.940097,0.950952,0.949520,0.933503,0.967537,0.968084,0.970380,0.954032
test,1.556776,0.657323,0.661081,0.655215,0.644590,0.619983,0.625489,0.620337,0.613171,0.695192,0.673997,0.665554,0.647251


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type AppInteractiveNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Dense. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GeLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MappedAlignment. It won't 

#### embedding concat

In [33]:
logging.info('start training')
app_interactive_net = train(train_app_ids, test_app_ids, AppConcat, AppListConfig())
# torch.save(app_interactive_net, 'app_interactive_net.model.torch')
logging.info('finish training')

2020-09-15 14:28:14,922 - INFO - start training


2020-09-15 14:35:58,618 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.246108,0.631134,0.640552,0.644655,0.623223,0.586166,0.604733,0.609480,0.593874,0.631077,0.628768,0.633509,0.612698
test,1.130675,0.596657,0.603135,0.604257,0.583222,0.552689,0.564052,0.572257,0.552421,0.564217,0.594346,0.589718,0.569659


2020-09-15 14:41:07,322 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.214417,0.704176,0.694916,0.688854,0.679213,0.653033,0.646985,0.643574,0.639674,0.689472,0.690332,0.679055,0.665157
test,1.123548,0.654015,0.652060,0.632731,0.623162,0.602245,0.604759,0.590226,0.584815,0.654020,0.653597,0.626495,0.611626


...

2020-09-15 15:49:13,964 - INFO - epoch : 14


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.286862,0.983713,0.989195,0.99137,0.983271,0.973126,0.983809,0.988168,0.979490,0.990249,0.991362,0.992834,0.983564
test,2.218584,0.680846,0.666347,0.65885,0.644891,0.641294,0.631018,0.619001,0.610658,0.716913,0.677174,0.675162,0.650530


2020-09-15 15:54:23,252 - INFO - epoch : 15


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.272406,0.983384,0.989905,0.992517,0.985209,0.972927,0.985752,0.990352,0.982350,0.990106,0.991159,0.993326,0.984940
test,2.344660,0.679181,0.664815,0.657383,0.643012,0.640799,0.630454,0.618597,0.609813,0.714076,0.674875,0.672864,0.648076


2020-09-15 15:54:23,271 - INFO - finish training


### userlog

In [32]:
userlog_ids = se_userlog_cross.index
train_userlog_ids = list( set(userlog_ids) & set(all_train_id) )
test_userlog_ids = list( set(userlog_ids) & set(all_test_id) )
len(train_userlog_ids), len(test_userlog_ids)

(253808, 65313)

In [33]:
logging.info('start training')
userlog_net = train(train_userlog_ids, test_userlog_ids, UserlogNetwork, Userlogconfig())
torch.save(userlog_net, 'userlog_net.model.torch')
logging.info('finish training')

2020-11-21 16:01:15,137 - INFO - start training


2020-11-21 16:07:55,787 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.123565,0.661081,0.687085,0.652670,0.614958,0.609075,0.690630,0.655232,0.611802,0.658727,0.712885,0.657810,0.606028
test,1.097841,0.666795,0.691883,0.645931,0.614864,0.636049,0.694681,0.655773,0.621365,0.686424,0.699343,0.637856,0.598229


2020-11-21 16:14:37,752 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.121829,0.696853,0.705701,0.669636,0.653964,0.679414,0.709764,0.678360,0.669126,0.732564,0.733343,0.679199,0.649394
test,1.096075,0.685578,0.697685,0.651944,0.628321,0.667414,0.699101,0.665345,0.646203,0.718951,0.705812,0.641953,0.612117


...

2020-11-21 17:15:05,516 - INFO - epoch : 10


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.657860,0.922429,0.923828,0.917746,0.900691,0.892254,0.910911,0.899500,0.887097,0.947103,0.939830,0.932570,0.910122
test,1.385011,0.704068,0.703649,0.653560,0.638083,0.678864,0.684357,0.631678,0.625078,0.736484,0.716488,0.661117,0.638019


2020-11-21 17:21:55,288 - INFO - epoch : 11


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.624044,0.930612,0.930695,0.925338,0.907356,0.900928,0.919259,0.911084,0.895207,0.951879,0.943924,0.933775,0.913603
test,1.471649,0.701179,0.701440,0.650861,0.636701,0.676004,0.681331,0.627185,0.622550,0.731727,0.714116,0.658968,0.637467


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UserlogNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LayerNorm. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MLPAttentionPool. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Dense. It won't b

In [28]:
logging.info('start training')
userlog_net = train(train_userlog_ids, test_userlog_ids, UserlogNetwork, Userlogconfig())
torch.save(userlog_net, 'userlog_net.time.model.torch')
logging.info('finish training')

2020-09-22 10:34:08,121 - INFO - start training


2020-09-22 10:40:40,810 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.140994,0.639166,0.671704,0.636347,0.619618,0.61401,0.673804,0.639686,0.641926,0.648556,0.692170,0.634426,0.611097
test,1.108052,0.654110,0.682417,0.633664,0.619566,0.62988,0.677721,0.635874,0.630114,0.676797,0.690993,0.623757,0.604759


2020-09-22 10:47:17,798 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.112639,0.688267,0.697222,0.661153,0.641434,0.667679,0.707057,0.674506,0.659372,0.710439,0.726207,0.665881,0.631977
test,1.095120,0.681295,0.693576,0.646917,0.625297,0.664957,0.697023,0.659643,0.642201,0.718831,0.704503,0.639885,0.611411


...

2020-09-22 11:47:53,110 - INFO - epoch : 10


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.637309,0.929817,0.930683,0.926508,0.906249,0.900614,0.914673,0.907474,0.888936,0.949019,0.946639,0.942489,0.915194
test,1.390441,0.705951,0.691267,0.654438,0.636215,0.677309,0.668242,0.634455,0.625857,0.742770,0.709132,0.661909,0.633918


2020-09-22 11:54:42,071 - INFO - epoch : 11


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.617304,0.931445,0.935663,0.931222,0.910254,0.901383,0.920246,0.912916,0.891772,0.949004,0.949997,0.945474,0.919417
test,1.487047,0.706345,0.691662,0.654836,0.636245,0.678031,0.668020,0.634325,0.625137,0.742415,0.710281,0.662935,0.635119


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UserlogNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LayerNorm. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MLPAttentionPool. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Dense. It won't b

### multi-view origin

In [18]:
logging.info('start training')
multiview_net = train(all_train_id, all_test_id, MaskMultiViewNetwork, MultiviewConfig())
logging.info('finish training')


2020-08-18 12:35:21,881 - INFO - start training


2020-08-18 12:44:56,178 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.13153,0.706510,0.708371,0.685238,0.670495,0.670917,0.680005,0.662705,0.653633,0.729398,0.720218,0.682488,0.658590
test,1.07665,0.695295,0.694480,0.663009,0.650522,0.659466,0.671490,0.647814,0.638845,0.734571,0.702502,0.657258,0.640783


2020-08-18 12:54:27,792 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.133015,0.710420,0.715248,0.689075,0.672711,0.669171,0.677196,0.659510,0.650319,0.765478,0.747288,0.699105,0.669898
test,1.093422,0.690579,0.695367,0.661705,0.649189,0.659655,0.673506,0.647572,0.637842,0.723562,0.707220,0.658730,0.642798


...

,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.074400,0.744134,0.749875,0.724856,0.712872,0.689308,0.704271,0.683912,0.673346,0.786552,0.779379,0.734304,0.715091
test,1.061564,0.710964,0.715482,0.683907,0.673232,0.662330,0.679748,0.653021,0.644705,0.744490,0.730880,0.686537,0.670843


KeyboardInterrupt: 

Exception ignored in: 'pandas._libs.lib.c_is_list_like'
Traceback (most recent call last):
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/abc.py", line 190, in __instancecheck__
    subclass in cls._abc_negative_cache):
KeyboardInterrupt
2020-08-18 14:18:27,902 - ERROR - Internal Python error in the inspect module.
Below is the traceback from this internal error.

2020-08-18 14:18:27,905 - INFO - 
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-4d8cf1d49425>", line 2, in <module>
    multiview_net = train(all_train_id, all_test_id, MaskMultiViewNetwork, MultiviewConfig())
  File "<ipython-input-12-bf89350227df>", line 111, in train
    loss.backward()
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/tensor.py", line 107, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/autograd/__init__.py", line 93, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/huangweilin/anaconda3/envs/hyr/lib/python3.6

KeyboardInterrupt: 

### multi-view fill zeros

In [19]:
logging.info('start training')
multiview_net_fill_zeros = train(all_train_id, all_test_id, MaskMultiViewNetworkFillZero, MultiviewConfig())
torch.save(multiview_net_fill_zeros, 'multiview_net.model.torch')

logging.info('finish training')


2020-09-14 21:34:14,248 - INFO - start training


2020-09-14 21:49:58,854 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.111856,0.711739,0.702694,0.687779,0.668809,0.659636,0.666758,0.660282,0.635521,0.695571,0.705527,0.684604,0.659332
test,1.088339,0.683791,0.679919,0.664193,0.645846,0.640413,0.657332,0.647237,0.624773,0.706432,0.684996,0.662938,0.642265


2020-09-14 22:11:16,513 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.093747,0.736497,0.734152,0.709063,0.698224,0.683341,0.697493,0.677141,0.666931,0.739088,0.756504,0.707727,0.698540
test,1.066064,0.703346,0.712843,0.677866,0.672511,0.659944,0.686868,0.655035,0.649527,0.732043,0.727738,0.678340,0.673593


...

2020-09-15 00:23:25,839 - INFO - epoch : 14


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.499395,0.948907,0.958657,0.959038,0.947144,0.920148,0.943468,0.945199,0.930788,0.964418,0.970390,0.967267,0.953655
test,1.637074,0.716626,0.704921,0.678116,0.672046,0.665636,0.666193,0.637884,0.636799,0.760700,0.721078,0.689234,0.671295


2020-09-15 00:33:21,483 - INFO - epoch : 15


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.475660,0.954191,0.962427,0.962540,0.952318,0.928649,0.949510,0.950479,0.938182,0.970698,0.971797,0.969223,0.957690
test,1.772926,0.716069,0.705605,0.677899,0.671672,0.664578,0.666417,0.637947,0.637080,0.760254,0.723760,0.690125,0.671683


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MaskMultiViewNetworkFillZero. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MaskMultiViewEncoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UserNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of ty

In [18]:
logging.info('start training')
multiview_net_fill_zeros_in = train(all_train_id, all_test_id, MaskMultiViewNetworkFillZeroIn, MultiviewAppInteractiveConfig())
torch.save(multiview_net_fill_zeros_in, 'multiview_net_fill_zeros_in.model.torch')
logging.info('finish training')



2020-08-29 18:34:37,516 - INFO - start training


2020-08-29 18:48:09,526 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.126358,0.705215,0.710719,0.679624,0.666651,0.662787,0.683622,0.659230,0.650618,0.721982,0.728879,0.675969,0.656210
test,1.090101,0.690278,0.702380,0.668095,0.662796,0.649810,0.677540,0.647924,0.644375,0.723086,0.713270,0.666466,0.658047


2020-08-29 19:01:41,313 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.096660,0.724189,0.727441,0.706601,0.698606,0.675925,0.682094,0.670071,0.668738,0.739325,0.749852,0.711015,0.700512
test,1.070329,0.696866,0.710250,0.680571,0.675359,0.651993,0.675385,0.649888,0.649723,0.728813,0.720176,0.684157,0.676089


...

2020-08-29 21:56:40,421 - INFO - epoch : 14


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.469882,0.952821,0.965083,0.965593,0.948239,0.923864,0.949036,0.951040,0.933856,0.968279,0.975082,0.973897,0.952696
test,1.541320,0.722306,0.714199,0.686216,0.677457,0.668443,0.665014,0.638514,0.638987,0.754699,0.733541,0.699673,0.680820


2020-08-29 22:10:06,157 - INFO - epoch : 15


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.452085,0.959157,0.967296,0.968399,0.951116,0.932484,0.954289,0.956001,0.934998,0.980017,0.974008,0.974967,0.959265
test,1.632909,0.720133,0.713392,0.685645,0.676791,0.667648,0.665212,0.639261,0.639450,0.752750,0.732778,0.699674,0.680962


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MaskMultiViewNetworkFillZeroIn. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MaskMultiViewEncoderAppInteractive. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UserNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for

### multi-view generate

In [45]:
logging.info('start training')
multiview_net = train(all_train_id, all_test_id, MaskMultiViewNetworkGenerate, MultiviewConfig())
logging.info('finish training')


2020-08-18 21:08:33,529 - INFO - start training


2020-08-18 21:18:28,108 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.139680,0.706377,0.713519,0.681733,0.670062,0.679534,0.686952,0.664742,0.661515,0.729414,0.739321,0.683441,0.659703
test,1.089786,0.695458,0.700460,0.665199,0.649478,0.660650,0.676196,0.650075,0.639806,0.741551,0.714471,0.662504,0.640281


2020-08-18 21:28:21,828 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.090567,0.733777,0.734486,0.708648,0.692956,0.686946,0.695310,0.677156,0.671860,0.767480,0.754646,0.709362,0.678106
test,1.083644,0.706365,0.707454,0.673748,0.661567,0.661665,0.674832,0.650041,0.640682,0.745974,0.719726,0.670033,0.655147


...

2020-08-18 22:56:33,697 - INFO - epoch : 10


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.636568,0.923907,0.934465,0.930690,0.913420,0.883680,0.910934,0.904516,0.885633,0.957090,0.952228,0.947184,0.926109
test,1.380672,0.728654,0.709215,0.685146,0.665618,0.677575,0.667086,0.645780,0.632471,0.769372,0.724895,0.691950,0.663090


2020-08-18 23:06:22,537 - INFO - epoch : 11


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.614675,0.927636,0.939902,0.936026,0.920050,0.890355,0.918411,0.913039,0.896480,0.953963,0.955822,0.950152,0.929230
test,1.470782,0.724908,0.703994,0.681231,0.661731,0.674845,0.662443,0.641937,0.628719,0.762828,0.718483,0.687675,0.658327


2020-08-18 23:06:22,557 - INFO - finish training


### multi-view generate with residual

In [38]:
logging.info('start training')
multiview_net = train(all_train_id, all_test_id, MaskMultiViewNetworkGenerateWithResidual, MultiviewConfig())
logging.info('finish training')

2020-08-28 14:48:30,292 - INFO - start training


2020-08-28 14:59:16,887 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.113564,0.714443,0.718602,0.692679,0.681180,0.6611,0.680333,0.661022,0.649226,0.714576,0.737856,0.694081,0.679352
test,1.081254,0.685903,0.696107,0.662904,0.656415,0.6438,0.670798,0.644628,0.635145,0.705631,0.704958,0.660087,0.654929


2020-08-28 15:09:32,472 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.083144,0.739054,0.736637,0.710678,0.701953,0.694345,0.705544,0.684860,0.673756,0.763963,0.758780,0.713857,0.698352
test,1.065084,0.714288,0.721208,0.685568,0.678597,0.671362,0.693574,0.664061,0.653611,0.749971,0.734398,0.683972,0.675371


...

2020-08-28 16:41:11,348 - INFO - epoch : 10


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.560010,0.942353,0.950592,0.946967,0.930253,0.910814,0.935083,0.931959,0.913953,0.965711,0.963136,0.956768,0.934712
test,1.478623,0.729027,0.710515,0.687826,0.677420,0.682640,0.667350,0.647498,0.641689,0.760419,0.729835,0.698435,0.678245


2020-08-28 16:51:19,387 - INFO - epoch : 11


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.535913,0.943258,0.954341,0.953130,0.937197,0.913620,0.939778,0.938334,0.919435,0.962271,0.965600,0.962672,0.943763
test,1.600581,0.729612,0.712379,0.690496,0.680301,0.682921,0.668264,0.648765,0.643159,0.760645,0.732237,0.701406,0.681603


2020-08-28 16:51:19,403 - INFO - finish training


### multi-view generate with residual App interactive

In [23]:
logging.info('start training')
multiview_net = train(all_train_id, all_test_id, MaskMultiViewNetworkGenerateWithResidualAppInteractive, MultiviewAppInteractiveConfig())
logging.info('finish training') 


2020-09-14 12:22:46,209 - INFO - start training


2020-09-14 12:36:13,660 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.118288,0.701012,0.707228,0.677716,0.666562,0.667627,0.679594,0.651482,0.639915,0.722677,0.729148,0.684292,0.666779
test,1.086685,0.691058,0.693461,0.659199,0.655040,0.656158,0.671357,0.642192,0.634545,0.725455,0.701278,0.654524,0.652684


2020-09-14 12:49:45,945 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.108478,0.727602,0.731444,0.705261,0.690548,0.688575,0.706686,0.681559,0.658097,0.762080,0.754349,0.715053,0.694214
test,1.075691,0.697608,0.710853,0.679232,0.669939,0.664508,0.690802,0.662799,0.647939,0.740442,0.727415,0.681899,0.668189


...

2020-09-14 15:45:42,317 - INFO - epoch : 14


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.471807,0.953888,0.963115,0.962886,0.948215,0.926580,0.948727,0.948196,0.934604,0.974604,0.975040,0.973183,0.950645
test,1.563570,0.736768,0.724177,0.699705,0.682952,0.680639,0.677953,0.653694,0.645646,0.776329,0.742009,0.710395,0.680617


2020-09-14 15:59:14,072 - INFO - epoch : 15


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.455391,0.957108,0.965673,0.966516,0.952188,0.931178,0.950920,0.953179,0.936842,0.97734,0.978701,0.976724,0.957686
test,1.652475,0.735876,0.725115,0.699778,0.682009,0.680578,0.679964,0.654624,0.645478,0.77720,0.743461,0.711054,0.680003


2020-09-14 15:59:14,094 - INFO - finish training


In [22]:
#with user log time
logging.  info('start training')
multiview_net = train(all_train_id, all_test_id, MaskMultiViewNetworkGenerateWithResidualAppInteractive, MultiviewAppInteractiveConfig())
logging.info('finish training') 


2020-09-25 17:03:06,348 - INFO - start training


2020-09-25 17:18:21,181 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.128388,0.720438,0.716076,0.690422,0.679212,0.678149,0.692345,0.669742,0.655000,0.746284,0.73371,0.686570,0.665371
test,1.090776,0.696940,0.702039,0.665875,0.657531,0.653479,0.679834,0.649221,0.639059,0.733011,0.71211,0.657377,0.642364


2020-09-25 17:33:31,369 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.098235,0.746497,0.741642,0.718177,0.706415,0.700200,0.707509,0.686057,0.673833,0.763547,0.758512,0.718101,0.698882
test,1.069703,0.711179,0.715716,0.680158,0.672997,0.667999,0.687704,0.656132,0.648007,0.752627,0.729346,0.677463,0.666702


...

2020-09-25 20:50:32,759 - INFO - epoch : 14


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.382062,0.967433,0.976273,0.978786,0.966655,0.942154,0.963480,0.967876,0.954892,0.985129,0.98480,0.985584,0.970138
test,1.717852,0.739158,0.724268,0.700255,0.683764,0.683402,0.676179,0.656714,0.648794,0.786152,0.74579,0.711596,0.683278


2020-09-25 21:05:41,379 - INFO - epoch : 15


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.369929,0.967340,0.977445,0.979818,0.968347,0.942775,0.964217,0.967732,0.954675,0.982390,0.986394,0.987311,0.973567
test,1.841868,0.736045,0.721228,0.697371,0.682700,0.679050,0.672266,0.652899,0.647438,0.782934,0.741985,0.707963,0.681610


2020-09-25 21:05:41,394 - INFO - finish training


In [2]:
a = [1, 2, 3, 4, 5]
a[::-1]

[5, 4, 3, 2, 1]

### drop all miss data

In [27]:
select_train_id = list(set(se_id_install_list.index) & set(df_install_behave.index) & set(se_userlog_cross.index) & set(all_train_id))
select_test_id = list(set(se_id_install_list.index) & set(df_install_behave.index) & set(se_userlog_cross.index) & set(all_test_id))
len(select_train_id), len(select_test_id)

(97077, 29931)

In [28]:
class MultiviewConfig():
    _instance = None
    def __new__(cls, *args, **kw):
        if cls._instance is None:
            cls._instance = object.__new__(cls, *args, **kw)
        return cls._instance
    
    def __init__(self):
        super().__init__()
        self.hidden = 256
#         self.app_list_net = torch.load('app_list_net.model.torch')
#         self.app_behave_net =  torch.load('app_list_net.model.torch')
#         self.user_net =  torch.load('user_net.model.torch')
        self.app_list_net = AppListNetwork(AppListConfig())
        self.app_behave_net = AppBehaveNetwork(AppbehaveConfig())
        self.user_net = UserNetwork(UserNetConfig())
        self.userlog_network = UserlogNetwork(Userlogconfig())

# torch.save(MultiviewConfig(), '../data_sortout/MultiviewConfig.torch')
logging.info('start training')
multiview_net = train(select_train_id, select_test_id, MaskMultiViewNetworkGenerateWithResidual, MultiviewConfig())
logging.info('finish training')


2020-08-22 17:46:40,734 - INFO - start training


2020-08-22 17:50:27,610 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.160160,0.738184,0.754198,0.731259,0.72146,0.675900,0.710604,0.690885,0.688659,0.765475,0.773854,0.735208,0.715432
test,1.095879,0.712147,0.724128,0.690805,0.67974,0.654793,0.690226,0.659012,0.651486,0.739114,0.729510,0.686355,0.670619


2020-08-22 17:54:14,249 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.151155,0.757091,0.768314,0.744837,0.734631,0.707345,0.735962,0.714326,0.710076,0.777523,0.785876,0.745775,0.726236
test,1.099943,0.727262,0.736574,0.700015,0.689821,0.676333,0.708481,0.675446,0.668193,0.770310,0.744973,0.694324,0.679624


...

2020-08-22 18:28:25,225 - INFO - epoch : 10


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.271222,0.982059,0.992975,0.994920,0.987077,0.965133,0.989008,0.992511,0.982745,0.986815,0.994173,0.995267,0.987141
test,1.943805,0.729285,0.733624,0.706264,0.695259,0.678844,0.692535,0.665212,0.662107,0.751053,0.744943,0.711711,0.690235


2020-08-22 18:32:14,287 - INFO - epoch : 11


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.244150,0.984266,0.994514,0.996028,0.988942,0.969164,0.991792,0.994436,0.985426,0.989102,0.995406,0.996257,0.989018
test,2.087319,0.728412,0.733354,0.704129,0.693612,0.680950,0.693795,0.664198,0.660419,0.750780,0.745105,0.709498,0.689248


2020-08-22 18:32:14,303 - INFO - finish training


### handle missing view with less data

In [29]:
def reduce_set_number(id_set):
    not_missing_rate = len(id_set) / df_master_records.shape[0]
    return set(random.sample(id_set, int(len(id_set) * not_missing_rate)))

install_behave_set = set(df_install_behave.index) & (set(all_train_id) | set(all_test_id))
install_list_set = set(se_id_install_list.index) & (set(all_train_id) | set(all_test_id))
user_info_set = set(df_user_one_hot.index) & (set(all_train_id) | set(all_test_id))
user_log_set = set(se_userlog_cross.index) & (set(all_train_id) | set(all_test_id))

install_behave_set = reduce_set_number(install_behave_set)
install_list_set = reduce_set_number(install_list_set)
user_info_set = reduce_set_number(user_info_set)
user_log_set = reduce_set_number(user_log_set)

logging.info('start training')
multiview_net = train(select_train_id, select_test_id, MaskMultiViewNetworkGenerateWithResidual, MultiviewConfig())
logging.info('finish training')

install_behave_set = set(df_install_behave.index) & (set(all_train_id) | set(all_test_id))
install_list_set = set(se_id_install_list.index) & (set(all_train_id) | set(all_test_id))
user_info_set = set(df_user_one_hot.index) & (set(all_train_id) | set(all_test_id))
user_log_set = set(se_userlog_cross.index) & (set(all_train_id) | set(all_test_id))


2020-08-22 18:32:15,374 - INFO - start training


2020-08-22 18:35:48,703 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.207349,0.695610,0.714183,0.689061,0.681934,0.663753,0.695145,0.677278,0.674955,0.732750,0.736045,0.691850,0.676528
test,1.130109,0.666371,0.684158,0.651632,0.645407,0.640294,0.672345,0.642235,0.635623,0.702657,0.694200,0.653739,0.646284


2020-08-22 18:39:17,104 - INFO - epoch : 1


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,1.189092,0.725164,0.734363,0.708201,0.700049,0.676039,0.702583,0.680886,0.678252,0.743834,0.743148,0.702470,0.688547
test,1.114723,0.687041,0.694087,0.659360,0.649103,0.642348,0.668988,0.638627,0.629852,0.723278,0.699957,0.654132,0.642187


...

2020-08-22 19:10:43,705 - INFO - epoch : 10


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.582574,0.945962,0.954728,0.953588,0.940042,0.920304,0.942939,0.945127,0.932056,0.951718,0.960274,0.955331,0.938725
test,1.661662,0.670530,0.675234,0.650203,0.643987,0.637855,0.649311,0.624134,0.623685,0.686209,0.680936,0.653317,0.638931


2020-08-22 19:14:16,129 - INFO - epoch : 11


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
train,0.551268,0.949345,0.958856,0.957938,0.946382,0.925246,0.949738,0.95045,0.939190,0.957400,0.963126,0.960341,0.945974
test,1.765508,0.668263,0.676200,0.651513,0.646158,0.638902,0.653661,0.62876,0.628239,0.688256,0.682070,0.654761,0.642122


2020-08-22 19:14:16,145 - INFO - finish training


In [34]:
start = time.time() 

In [35]:
time.time()  - start

7.752150774002075

### multi-view mean

In [34]:
user_net = torch.load('user_net.model.torch')
app_behave_net = torch.load('app_behave_net.model.torch')
app_list_net = torch.load('app_list_net.model.torch')
userlog_net = torch.load('userlog_net.model.torch')

In [35]:
train_user_ids, test_user_ids = all_train_id, all_test_id

app_behave_ids = list(df_install_behave.index)
train_app_behave_ids = list( set(app_behave_ids) & set(all_train_id) )
test_app_behave_ids = list( set(app_behave_ids) & set(all_test_id) )

app_list_ids = se_id_install_list.index
train_app_list_ids = list( set(app_list_ids) & set(all_train_id) )
test_app_list_ids = list( set(app_list_ids) & set(all_test_id) )

userlog_ids = se_userlog_cross.index
train_userlog_ids = list( set(userlog_ids) & set(all_train_id) )
test_userlog_ids = list( set(userlog_ids) & set(all_test_id) )

In [36]:
def predict_batch(model, master_ids):
    
    if(len(master_ids) > args.n_eval):
        master_ids = random.sample(master_ids, args.n_eval)

    torch_dataset = AppDataset(master_ids)
    data_loader = Data.DataLoader(
        dataset=torch_dataset,      
        batch_size=args.batch_size,      
        shuffle=False,
        collate_fn=collate_fn,
        num_workers = args.n_worker,
    )
    
    loss_list = []
    y1_list, y2_list, y3_list, y4_list = [], [], [], []
    label1_list, label2_list, label3_list, label4_list = [], [], [], []
#     for step, data in enumerate(tqdm(data_loader)):
    for step, data in enumerate(data_loader):
        loss, y1, y2, y3, y4, _ = model(data)
        
        loss_list.append(loss.item())
        y1_list.append(y1.cpu().detach().numpy())
        y2_list.append(y2.cpu().detach().numpy())
        y3_list.append(y3.cpu().detach().numpy())
        y4_list.append(y4.cpu().detach().numpy())

        label1_list.append(data['labels1'].cpu().detach().numpy())
        label2_list.append(data['labels2'].cpu().detach().numpy())
        label3_list.append(data['labels3'].cpu().detach().numpy())
        label4_list.append(data['labels4'].cpu().detach().numpy())

    loss = np.mean(loss_list)
    
    y1_np = np.concatenate(y1_list,  axis = 0)
    y2_np = np.concatenate(y2_list,  axis = 0)
    y3_np = np.concatenate(y3_list,  axis = 0)
    y4_np = np.concatenate(y4_list,  axis = 0)
    
    y1_np = softmax(y1_np, axis=1)
    y2_np = softmax(y2_np, axis=1)
    y3_np = softmax(y3_np, axis=1)
    y4_np = softmax(y4_np, axis=1)

    return y1_np, y2_np, y3_np, y4_np

In [37]:
user_info_y1, user_info_y2, user_info_y3, user_info_y4 = predict_batch(user_net.to(args.device), test_user_ids)
app_behave_y1, app_behave_y2, app_behave_y3, app_behave_y4 = predict_batch(app_behave_net.to(args.device), test_app_behave_ids)
app_list_y1, app_list_y2, app_list_y3, app_list_y4 = predict_batch(app_list_net.to(args.device), test_app_list_ids)
user_log_y1, user_log_y2, user_log_y3, user_log_y4 = predict_batch(userlog_net.to(args.device), test_userlog_ids)

In [38]:
df_score = pd.DataFrame(np.zeros((len(all_test_id), 17)) - 1,
                        columns = ['n_view', 
                                   'user_info_y1', 'user_info_y2', 'user_info_y3', 'user_info_y4',
                                   'app_behave_y1', 'app_behave_y2', 'app_behave_y3', 'app_behave_y4',
                                   'app_list_y1', 'app_list_y2', 'app_list_y3', 'app_list_y4',
                                   'user_log_y1', 'user_log_y2', 'user_log_y3', 'user_log_y4',],
                        index = all_test_id)

In [39]:
for master_id in test_user_ids:
    cnt = (master_id in install_behave_set) + (master_id in install_list_set) + (master_id in user_info_set) + (master_id in user_log_set)
    df_score.at[master_id, 'n_view'] = cnt

In [40]:
for master_id, y1, y2, y3, y4 in zip(test_user_ids, user_info_y1, user_info_y2, user_info_y3, user_info_y4):
    df_score.at[master_id, 'user_info_y1'] = y1[1]
    df_score.at[master_id, 'user_info_y2'] = y2[1]
    df_score.at[master_id, 'user_info_y3'] = y3[1]
    df_score.at[master_id, 'user_info_y4'] = y4[1]

for master_id, y1, y2, y3, y4 in zip(test_app_behave_ids, app_behave_y1, app_behave_y2, app_behave_y3, app_behave_y4):
    df_score.at[master_id, 'app_behave_y1'] = y1[1]
    df_score.at[master_id, 'app_behave_y2'] = y2[1]
    df_score.at[master_id, 'app_behave_y3'] = y3[1]
    df_score.at[master_id, 'app_behave_y4'] = y4[1]

for master_id, y1, y2, y3, y4 in zip(test_app_list_ids, app_list_y1, app_list_y2, app_list_y3, app_list_y4):
    df_score.at[master_id, 'app_list_y1'] = y1[1]
    df_score.at[master_id, 'app_list_y2'] = y2[1]
    df_score.at[master_id, 'app_list_y3'] = y3[1]
    df_score.at[master_id, 'app_list_y4'] = y4[1]

for master_id, y1, y2, y3, y4 in zip(test_userlog_ids, user_log_y1, user_log_y2, user_log_y3, user_log_y4):
    df_score.at[master_id, 'user_log_y1'] = y1[1]
    df_score.at[master_id, 'user_log_y2'] = y2[1]
    df_score.at[master_id, 'user_log_y3'] = y3[1]
    df_score.at[master_id, 'user_log_y4'] = y4[1]

In [41]:
df_score

,n_view,user_info_y1,user_info_y2,user_info_y3,user_info_y4,app_behave_y1,app_behave_y2,app_behave_y3,app_behave_y4,app_list_y1,app_list_y2,app_list_y3,app_list_y4,user_log_y1,user_log_y2,user_log_y3,user_log_y4
16,4.0,0.043197,0.084722,0.101530,0.100314,2.217182e-05,0.000778,0.003541,0.013321,0.041149,0.079708,0.092312,0.107405,0.017663,0.040805,0.057829,0.073891
23,4.0,0.071227,0.134023,0.148657,0.163682,2.556974e-03,0.013124,0.025040,0.071472,0.061252,0.119574,0.132285,0.145278,0.009087,0.025027,0.036023,0.047836
29,4.0,0.031564,0.076404,0.090274,0.104015,2.106017e-02,0.027412,0.036555,0.036719,0.041352,0.083324,0.095895,0.110081,0.051382,0.096559,0.122639,0.142505
30,4.0,0.048644,0.102597,0.128984,0.155108,8.499006e-05,0.003109,0.004332,0.009433,0.057632,0.116912,0.134505,0.152921,0.010999,0.028726,0.041083,0.053317
47,4.0,0.040456,0.089445,0.103790,0.120138,1.230687e-05,0.000382,0.000774,0.004789,0.088851,0.159184,0.172115,0.186267,0.008385,0.023297,0.033496,0.044178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402812,3.0,0.038739,0.090234,0.098871,0.092263,-1.000000e+00,-1.000000,-1.000000,-1.000000,0.021769,0.044044,0.054406,0.066765,0.060657,0.153745,0.158872,0.169239
402814,2.0,0.028243,0.073364,0.090483,0.105388,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.120396,0.286334,0.297020,0.306594
402817,4.0,0.038558,0.097692,0.120735,0.146537,3.858477e-07,0.000101,0.000793,0.007481,0.045894,0.095152,0.107810,0.121571,0.061754,0.154699,0.158414,0.168183
402818,4.0,0.043053,0.098654,0.124026,0.152971,2.422698e-06,0.000158,0.000231,0.001908,0.055278,0.106994,0.121058,0.135567,0.048516,0.094105,0.155220,0.197187


In [42]:
df_score['n_view'].value_counts() / df_score.shape[0]

4.0    0.394369
2.0    0.349834
3.0    0.200986
1.0    0.054812
Name: n_view, dtype: float64

In [43]:
score_names = ['user_info_y1', 'user_info_y2', 'user_info_y3', 'user_info_y4',
               'app_behave_y1', 'app_behave_y2', 'app_behave_y3', 'app_behave_y4',
               'app_list_y1', 'app_list_y2', 'app_list_y3', 'app_list_y4',
               'user_log_y1', 'user_log_y2', 'user_log_y3', 'user_log_y4']

def mean_score(x):
    score1, score2, score3, score4 = 0, 0, 0, 0
    for name in ['user_info_y1', 'app_behave_y1', 'app_list_y1', 'user_log_y1']:
        if(x[name] != -1):
            score1 += x[name]
    for name in ['user_info_y2', 'app_behave_y2', 'app_list_y2', 'user_log_y2']:
        if(x[name] != -1):
            score2 += x[name]
    for name in ['user_info_y3', 'app_behave_y3', 'app_list_y3', 'user_log_y3']:
        if(x[name] != -1):
            score3 += x[name]
    for name in ['user_info_y4', 'app_behave_y4', 'app_list_y4', 'user_log_y4']:
        if(x[name] != -1):
            score4 += x[name]

    return pd.Series({
        'y1_mean' : score1 / x['n_view'],
        'y2_mean' : score2 / x['n_view'],
        'y3_mean' : score3 / x['n_view'],
        'y4_mean' : score4 / x['n_view'],
    })
    
df_mean_score = df_score.apply(mean_score, axis = 1)

In [44]:
df_mean_score

,y1_mean,y2_mean,y3_mean,y4_mean
16,0.025508,0.051503,0.063803,0.073733
23,0.036031,0.072937,0.085501,0.107067
29,0.036340,0.070925,0.086341,0.098330
30,0.029340,0.062836,0.077226,0.092695
47,0.034426,0.068077,0.077544,0.088843
...,...,...,...,...
402812,0.040388,0.096008,0.104050,0.109422
402814,0.074320,0.179849,0.193751,0.205991
402817,0.036551,0.086911,0.096938,0.110943
402818,0.036712,0.074978,0.100134,0.121908


In [45]:
def max_score(x):
    score1 = max(x[['user_info_y1', 'app_behave_y1', 'app_list_y1', 'user_log_y1']])
    score2 = max(x[['user_info_y2', 'app_behave_y2', 'app_list_y2', 'user_log_y2']])
    score3 = max(x[['user_info_y3', 'app_behave_y3', 'app_list_y3', 'user_log_y3']])
    score4 = max(x[['user_info_y4', 'app_behave_y4', 'app_list_y4', 'user_log_y4']])
    return pd.Series({
        'y1_max' : score1,
        'y2_max' : score2,
        'y3_max' : score3,
        'y4_max' : score4,
    })

df_max_score = df_score.apply(max_score, axis = 1)

In [46]:
new_ids = set(df_master_records[df_master_records['new_client']].index) & set(df_mean_score.index)
old_ids = set(df_master_records[~df_master_records['new_client']].index) & set(df_mean_score.index)

ret_dict_mean = {
    '1m30+' : roc_auc_score(df_master_records['target_1m30+'].loc[all_test_id], df_mean_score['y1_mean']),
    '2m30+' : roc_auc_score(df_master_records['target_2m30+'].loc[all_test_id], df_mean_score['y2_mean']),
    '3m30+' : roc_auc_score(df_master_records['target_3m30+'].loc[all_test_id], df_mean_score['y3_mean']),
    '4m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[all_test_id], df_mean_score['y4_mean']),
    'new_1m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[new_ids], df_mean_score['y1_mean'].loc[new_ids]),
    'new_2m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[new_ids], df_mean_score['y2_mean'].loc[new_ids]),
    'new_3m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[new_ids], df_mean_score['y3_mean'].loc[new_ids]),
    'new_4m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[new_ids], df_mean_score['y4_mean'].loc[new_ids]),
    'old_1m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[old_ids], df_mean_score['y1_mean'].loc[old_ids]),
    'old_2m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[old_ids], df_mean_score['y2_mean'].loc[old_ids]),
    'old_3m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[old_ids], df_mean_score['y3_mean'].loc[old_ids]),
    'old_4m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[old_ids], df_mean_score['y4_mean'].loc[old_ids]),
}


ret_dict_max = {
    '1m30+' : roc_auc_score(df_master_records['target_1m30+'].loc[all_test_id], df_max_score['y1_max']),
    '2m30+' : roc_auc_score(df_master_records['target_2m30+'].loc[all_test_id], df_max_score['y2_max']),
    '3m30+' : roc_auc_score(df_master_records['target_3m30+'].loc[all_test_id], df_max_score['y3_max']),
    '4m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[all_test_id], df_max_score['y4_max']),
    'new_1m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[new_ids], df_max_score['y1_max'].loc[new_ids]),
    'new_2m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[new_ids], df_max_score['y2_max'].loc[new_ids]),
    'new_3m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[new_ids], df_max_score['y3_max'].loc[new_ids]),
    'new_4m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[new_ids], df_max_score['y4_max'].loc[new_ids]),
    'old_1m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[old_ids], df_max_score['y1_max'].loc[old_ids]),
    'old_2m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[old_ids], df_max_score['y2_max'].loc[old_ids]),
    'old_3m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[old_ids], df_max_score['y3_max'].loc[old_ids]),
    'old_4m30+' : roc_auc_score(df_master_records['target_4m30+'].loc[old_ids], df_max_score['y4_max'].loc[old_ids]),
}

pd.DataFrame([ret_dict_mean, ret_dict_max], index = ['mean', 'max'])

,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
mean,0.704300,0.698186,0.668765,0.656973,0.628328,0.637894,0.636384,0.637431,0.636633,0.646105,0.652943,0.659411
max,0.658443,0.661585,0.636244,0.627843,0.599650,0.615471,0.614774,0.616238,0.600646,0.618958,0.626533,0.633462


### train all & test single

In [19]:
def extend(origin_data, start_extend_id, add_id = True):
    train_data = origin_data.loc[list(set(all_train_id) & set(origin_data.index))]
    extend_id = list(range(start_extend_id, start_extend_id+train_data.shape[0]))
    if add_id:
        start_extend_id += train_data.shape[0]
    
    new_feature = train_data.copy()
    new_label = df_target.loc[train_data.index].copy()
    
    new_feature.index = extend_id
    new_label.index = extend_id
    
    return new_feature, new_label, start_extend_id

start_extend_id = max(df_master_records.index) + 1
extend_user_info, extend_user_info_label, start_extend_id =  extend(df_user_one_hot, start_extend_id)
extend_applist, extend_applist_label, start_extend_id =  extend(se_id_install_list, start_extend_id)
extend_appbehave, extend_appbehave_label, start_extend_id =  extend(df_install_behave, start_extend_id, False)
extend_appbehavetime, extend_appbehavetime_label, start_extend_id =  extend(df_behave_time, start_extend_id)
extend_userlog, extend_userlog_label, start_extend_id =  extend(se_userlog_cross, start_extend_id)

df_user_one_hot = df_user_one_hot.append(extend_user_info)
se_id_install_list = se_id_install_list.append(extend_applist)
df_install_behave = df_install_behave.append(extend_appbehave)
df_behave_time = df_behave_time.append(extend_appbehavetime)
se_userlog_cross = se_userlog_cross.append(extend_userlog)

df_target = df_target.append(extend_user_info_label)
df_target = df_target.append(extend_applist_label)
df_target = df_target.append(extend_appbehave_label)
df_target = df_target.append(extend_userlog_label)
extend_train_ids = list(range(max(df_master_records.index) + 1, max(df_target.index) + 1)) + all_train_id

In [20]:
install_behave_set = set(df_install_behave.index) & (set(extend_train_ids) | set(all_test_id))
install_list_set = set(se_id_install_list.index) & (set(extend_train_ids) | set(all_test_id))
user_info_set = set(df_user_one_hot.index) & (set(extend_train_ids) | set(all_test_id))
user_log_set = set(se_userlog_cross.index) & (set(extend_train_ids) | set(all_test_id))

In [21]:
USE_VIEW = ""
def collate_fn_single(master_ids):
    
    name = USE_VIEW
    master_ids = np.array(master_ids)

#     sub_master_id = se_id_install_list.loc[master_ids]
#     df_sub_behave = df_install_behave.loc[master_ids]
#     df_sub_time = df_behave_time.loc[master_ids]
    for i, master_id in enumerate(master_ids):
        
        if master_id in user_info_set and name == 'user_info':
            x_dict['user_info'][i] = df_user_one_hot.loc[master_id].values
            x_dict['view_mask'][i][0] = 1
        else:
            x_dict['user_info'][i] = 0
            x_dict['view_mask'][i][0] = 0

        if master_id in install_list_set and name == 'install_list':
            app_list = se_id_install_list.at[master_id][:args.app_install_list_max_length]
            x_dict['app_list_len'][i] = len(app_list) + 1
            x_dict['app_list'][i][1 : x_dict['app_list_len'][i]] = app_list
            x_dict['app_list'][i][x_dict['app_list_len'][i] :] = 0
            x_dict['view_mask'][i][1] = 1
        else:
            x_dict['app_list_len'][i] = 1
            x_dict['app_list'][i] = 0
            x_dict['view_mask'][i][1] = 0

        if master_id in install_behave_set and name == 'install_behave':
            app_behave = df_install_behave['pkg_id'].at[master_id][-args.app_behave_max_length:]
            len_app = len(app_behave) + 1
            x_dict['app_behave_len'][i] = len_app
            x_dict['app_behave'][i][1: len_app] = app_behave
            x_dict['app_behave'][i][len_app :] = 0

            x_dict['app_behave_time_cut'][i][1:len_app] = df_behave_time['cut_id'].at[master_id][-args.app_behave_max_length:]
            x_dict['app_behave_time_qcut'][i][1:len_app] = df_behave_time['qcut_id'].at[master_id][-args.app_behave_max_length:]
            x_dict['app_behave_action'][i][1:len_app] = df_install_behave['action'].at[master_id][-args.app_behave_max_length:]
            x_dict['view_mask'][i][2] = 1
        else:
            x_dict['app_behave_len'][i] = 1
            x_dict['app_behave'][i] = 0
            x_dict['app_behave'][i] = 0

            x_dict['app_behave_time_cut'][i] = 0
            x_dict['app_behave_time_qcut'][i] = 0
            x_dict['app_behave_action'][i] = 0
            x_dict['view_mask'][i][2] = 0
        
        
        if master_id in user_log_set and name == 'user_log':
            userlog_list = se_userlog_cross.at[master_id][:args.userlog_max_length]
            x_dict['userlog_len'][i] = len(userlog_list) + 1
            x_dict['userlog'][i][1 : x_dict['userlog_len'][i]] = userlog_list
            x_dict['userlog'][i][x_dict['userlog_len'][i] :] = 0
            x_dict['view_mask'][i][3] = 1
        else:
            x_dict['userlog_len'][i] = 1
            x_dict['userlog'][i] = 0
            x_dict['view_mask'][i][3] = 0
            
    len_id = master_ids.shape[0]
    x_dict['app_list'][len_id:] = 0
    x_dict['app_behave'][len_id:] = 0
    return {
        
        'user_info' : torch.tensor(x_dict['user_info'][:len_id]).float(),
        
        'app_list' : torch.tensor(x_dict['app_list'][:len_id]).long(),
        'app_list_te_qcut' : torch.tensor(x_dict['app_list_te_qcut'][:len_id]).long(),
        'app_list_len' : torch.tensor(x_dict['app_list_len'][:len_id]).long(),
        
        'app_behave' : torch.tensor(x_dict['app_behave'][:len_id]).long(),
        'app_behave_te_qcut' : torch.tensor(x_dict['app_behave_te_qcut'][:len_id]).long(),
        'app_behave_len' : torch.tensor(x_dict['app_behave_len'][:len_id]).long(),
        
        'app_behave_time_cut' : torch.tensor(x_dict['app_behave_time_cut'][:len_id]).long(),
        'app_behave_time_qcut' : torch.tensor(x_dict['app_behave_time_qcut'][:len_id]).long(),
        'app_behave_action' : torch.tensor(x_dict['app_behave_action'][:len_id]).long(),
        'userlog' : torch.tensor(x_dict['userlog'][:len_id]).long(),
        'userlog_len' : torch.tensor(x_dict['userlog_len'][:len_id]).long(),
        'view_mask' : torch.tensor(x_dict['view_mask'][:len_id]).long(),
        'labels1' : torch.tensor(df_target.loc[master_ids]['target_1m30+'].values).long(),
        'labels2' : torch.tensor(df_target.loc[master_ids]['target_2m30+'].values).long(),
        'labels3' : torch.tensor(df_target.loc[master_ids]['target_3m30+'].values).long(),
        'labels4' : torch.tensor(df_target.loc[master_ids]['target_4m30+'].values).long(),
    }
def eval_data_single(model, master_ids):
    
    if(len(master_ids) > args.n_eval):
        master_ids = random.sample(master_ids, args.n_eval)

    torch_dataset = AppDataset(master_ids)
    data_loader = Data.DataLoader(
        dataset=torch_dataset,      
        batch_size=args.batch_size,      
        shuffle=False,
        collate_fn=collate_fn_single,
        num_workers = args.n_worker,
    )
    
    loss_list = []
    y1_list, y2_list, y3_list, y4_list = [], [], [], []
    label1_list, label2_list, label3_list, label4_list = [], [], [], []
    for step, data in enumerate(tqdm(data_loader)):
#     for step, data in enumerate(data_loader):
        loss, y1, y2, y3, y4, _ = model(data)
        
        loss_list.append(loss.item())
        y1_list.append(y1.cpu().detach().numpy())
        y2_list.append(y2.cpu().detach().numpy())
        y3_list.append(y3.cpu().detach().numpy())
        y4_list.append(y4.cpu().detach().numpy())

        label1_list.append(data['labels1'].cpu().detach().numpy())
        label2_list.append(data['labels2'].cpu().detach().numpy())
        label3_list.append(data['labels3'].cpu().detach().numpy())
        label4_list.append(data['labels4'].cpu().detach().numpy())

    loss = np.mean(loss_list)
    
    y1_np = np.concatenate(y1_list,  axis = 0)
    y2_np = np.concatenate(y2_list,  axis = 0)
    y3_np = np.concatenate(y3_list,  axis = 0)
    y4_np = np.concatenate(y4_list,  axis = 0)

    labels1_np = np.concatenate(label1_list,  axis = 0)
    labels2_np = np.concatenate(label2_list,  axis = 0)
    labels3_np = np.concatenate(label3_list,  axis = 0)
    labels4_np = np.concatenate(label4_list,  axis = 0)

    auc1 = roc_auc_score(labels1_np, y1_np[:, 1])
    auc2 = roc_auc_score(labels2_np, y2_np[:, 1])
    auc3 = roc_auc_score(labels3_np, y3_np[:, 1])
    auc4 = roc_auc_score(labels4_np, y4_np[:, 1])
    auc_all = [auc1, auc2, auc3, auc4]
    
    new_client = np.array(df_master_records['new_client'].loc[master_ids].values)
    new_client_auc1 = roc_auc_score(labels1_np[new_client], y1_np[:, 1][new_client])
    new_client_auc2 = roc_auc_score(labels2_np[new_client], y2_np[:, 1][new_client])
    new_client_auc3 = roc_auc_score(labels3_np[new_client], y3_np[:, 1][new_client])
    new_client_auc4 = roc_auc_score(labels4_np[new_client], y4_np[:, 1][new_client])
    auc_new_client = [new_client_auc1, new_client_auc2, new_client_auc3, new_client_auc4]

    old_client = ~new_client
    old_client_auc1 = roc_auc_score(labels1_np[old_client], y1_np[:, 1][old_client])
    old_client_auc2 = roc_auc_score(labels2_np[old_client], y2_np[:, 1][old_client])
    old_client_auc3 = roc_auc_score(labels3_np[old_client], y3_np[:, 1][old_client])
    old_client_auc4 = roc_auc_score(labels4_np[old_client], y4_np[:, 1][old_client])
    auc_old_client = [old_client_auc1, old_client_auc2, old_client_auc3, old_client_auc4]

    return {
        'loss' : loss,
        '1m30+' : auc1,
        '2m30+' : auc2,
        '3m30+' : auc3,
        '4m30+' : auc4,
        'new_1m30+' : new_client_auc1,
        'new_2m30+' : new_client_auc2,
        'new_3m30+' : new_client_auc3,
        'new_4m30+' : new_client_auc4,
        'old_1m30+' : old_client_auc1,
        'old_2m30+' : old_client_auc2,
        'old_3m30+' : old_client_auc3,
        'old_4m30+' : old_client_auc4,
    }

def train_multi_test_signle(train_ids, test_ids, model_class, config):
        
    torch_dataset = AppDataset(train_ids)
    data_loader = Data.DataLoader(
        dataset=torch_dataset,      
        batch_size=args.batch_size,      
        shuffle=True,
        collate_fn=collate_fn,
        num_workers = args.n_worker,
    )
    
    model = model_class(config).to(args.device)
#     model = UserNetwork().to(args.device)
#     model = AppBehaveNetwork().to(args.device)
#     model = AppListNetwork().to(args.device)
#     model = MaskMultiViewNetwork(config).to(args.device)
#     optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.001)

    decay = ["app_network"]

    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in decay)], "weight_decay": args.weight_decay},
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr = args.lr, weight_decay = args.weight_decay)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=int(len(train_ids)//(args.batch_size)), num_training_steps=int(len(train_ids) / args.batch_size * args.epoch)
    )

    for epoch in range(args.epoch):
        model.train()
        for step, data in enumerate(tqdm(data_loader)):
            loss, y1, y2, y3, y4, _ = model(data)            
            #backward
            S = time.time()
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 2)
            optimizer.step()
            scheduler.step()
        
        model.eval()
#         train_ret_dict = eval_data(model, train_ids)
        test_ret_dict = eval_data(model, test_ids)
#         df_ret = pd.DataFrame([
#             train_ret_dict, test_ret_dict
#         ], index = ['train', 'test'])
        
        
        install_behave_test = list(set(install_behave_set) & set(test_ids))
        install_list_test = list(set(install_list_set) & set(test_ids))
        user_info_test = list(set(user_info_set) & set(test_ids))
        user_log_test = list(set(user_log_set) & set(test_ids))
        
        behave_ret_dict = eval_data(model, install_behave_test)
        list_ret_dict = eval_data(model, install_list_test)
        userinfo_ret_dict = eval_data(model, user_info_test)
        userlog_ret_dict = eval_data(model, user_log_test)

        df_ret = pd.DataFrame([
            test_ret_dict, behave_ret_dict, list_ret_dict, userinfo_ret_dict, userlog_ret_dict
        ], index = ['test', 'behave', 'list', 'user_info', 'user_log'])
        

        logging.info('epoch : %d' % epoch)
        ipd.display(df_ret)
        torch.save(model, 'model/multiview_net_extend.torch.model_%d' % epoch)

        
    return model

In [26]:
class MultiviewConfig():
    _instance = None
    def __new__(cls, *args, **kw):
        if cls._instance is None:
            cls._instance = object.__new__(cls, *args, **kw)
        return cls._instance
    
    def __init__(self):
        super().__init__()
        self.hidden = 256
#         self.app_list_net = torch.load('app_list_net.model.torch')
#         self.app_behave_net =  torch.load('app_list_net.model.torch')
#         self.user_net =  torch.load('user_net.model.torch')
        self.app_list_net = AppListNetwork(AppListConfig())
        self.app_behave_net = AppBehaveNetwork(AppbehaveConfig())
        self.user_net = UserNetwork(UserNetConfig())
        self.userlog_network = UserlogNetwork(Userlogconfig())
# torch.save(MultiviewConfig(), '../data_sortout/MultiviewConfig.torch')
logging.info('start training')
# sample_ids = random.sample(extend_train_ids, int(1e4))
multiview_net = train_multi_test_signle(extend_train_ids, all_test_id, MaskMultiViewNetworkGenerateWithResidualAppInteractive, MultiviewAppInteractiveConfig())
logging.info('finish training')


2020-09-16 16:58:40,195 - INFO - start training


2020-09-16 17:55:46,473 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
test,1.095009,0.691763,0.694753,0.654840,0.647161,0.651467,0.668614,0.638654,0.632369,0.732544,0.702787,0.644810,0.630804
behave,1.118676,0.687009,0.698860,0.663172,0.652225,0.634128,0.661118,0.635343,0.628042,0.735740,0.710433,0.656649,0.636682
list,1.107149,0.696179,0.705014,0.666723,0.657150,0.639259,0.668462,0.641331,0.633708,0.751102,0.711460,0.654671,0.638500
user_info,1.095398,0.691763,0.694753,0.654840,0.647161,0.651467,0.668614,0.638654,0.632369,0.732544,0.702787,0.644810,0.630804
user_log,1.101680,0.699591,0.703388,0.661796,0.653274,0.664055,0.683679,0.653573,0.646124,0.734221,0.708126,0.650319,0.636224


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MaskMultiViewNetworkGenerateWithResidualAppInteractive. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MaskMultiViewEncoderAppInteractive. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UserNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't 

2020-09-16 18:55:20,532 - INFO - epoch : 1


...

,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
test,1.933007,0.698785,0.685203,0.670367,0.652781,0.651932,0.646869,0.630921,0.619301,0.741396,0.703260,0.688074,0.656536
behave,2.257944,0.713879,0.699939,0.687438,0.667473,0.656626,0.655554,0.645439,0.632467,0.759606,0.715810,0.699726,0.666108
list,2.172117,0.713538,0.698423,0.686218,0.666821,0.658833,0.659660,0.648100,0.635979,0.758907,0.709625,0.697101,0.662982
user_info,1.933552,0.698785,0.685203,0.670367,0.652781,0.651932,0.646869,0.630921,0.619301,0.741396,0.703260,0.688074,0.656536
user_log,1.943625,0.720831,0.706203,0.689513,0.668873,0.683818,0.677923,0.658953,0.645758,0.754841,0.717781,0.700568,0.666987


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MaskMultiViewNetworkGenerateWithResidualAppInteractive. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MaskMultiViewEncoderAppInteractive. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UserNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't 

2020-09-17 05:56:00,676 - INFO - epoch : 15


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
test,2.082439,0.690380,0.679672,0.667269,0.650744,0.644672,0.642819,0.628427,0.617247,0.733798,0.697936,0.685893,0.656351
behave,2.478604,0.707720,0.695009,0.686130,0.665894,0.651870,0.652677,0.644617,0.630913,0.753523,0.709927,0.698571,0.665843
list,2.377302,0.705975,0.693314,0.684862,0.665025,0.652965,0.656418,0.647093,0.634125,0.752057,0.704254,0.696406,0.662967
user_info,2.083155,0.690380,0.679672,0.667269,0.650744,0.644672,0.642819,0.628427,0.617247,0.733798,0.697936,0.685893,0.656351
user_log,2.096937,0.712914,0.700454,0.685901,0.666225,0.676989,0.673284,0.655684,0.642606,0.748426,0.712511,0.698105,0.666181


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MaskMultiViewNetworkGenerateWithResidualAppInteractive. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MaskMultiViewEncoderAppInteractive. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UserNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't 

In [45]:
len(all_train_id)

326082

In [22]:
for i in range(12):
    
    model = torch.load('model/multiview_net_extend.torch.model_%d' % i)
    
    install_behave_test = list(set(install_behave_set) & set(all_test_id))
    install_list_test = list(set(install_list_set) & set(all_test_id))
    user_info_test = list(set(user_info_set) & set(all_test_id))
    user_log_test = list(set(user_log_set) & set(all_test_id))
    USE_VIEW = "install_behave"
    behave_ret_dict = eval_data_single(model, install_behave_test)
    USE_VIEW = "install_list"
    list_ret_dict = eval_data_single(model, install_list_test)
    USE_VIEW = "user_info"
    userinfo_ret_dict = eval_data_single(model, user_info_test)
    USE_VIEW = "user_log"
    userlog_ret_dict = eval_data_single(model, user_log_test)
    
    df_ret = pd.DataFrame([
        behave_ret_dict, list_ret_dict, userinfo_ret_dict, userlog_ret_dict
    ], index = ['behave', 'list', 'user_info', 'user_log'])
    
    ipd.display(df_ret)

,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.151112,0.610052,0.599398,0.597387,0.588470,0.553555,0.556810,0.556035,0.554620,0.642902,0.604329,0.600085,0.582040
list,1.143527,0.624394,0.617640,0.604201,0.591953,0.580899,0.588009,0.574195,0.566611,0.635012,0.610762,0.599161,0.581202
user_info,1.133006,0.599861,0.596399,0.583894,0.592625,0.604517,0.608624,0.596392,0.594478,0.637002,0.604368,0.587057,0.598192
user_log,1.097650,0.680743,0.692738,0.648655,0.627903,0.663718,0.689113,0.656076,0.637081,0.719800,0.698739,0.642320,0.609991


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.135440,0.625074,0.616569,0.608532,0.598819,0.574013,0.574133,0.569683,0.565470,0.654447,0.626361,0.613620,0.596644
list,1.145162,0.612551,0.601678,0.584761,0.576530,0.584695,0.583461,0.570575,0.565296,0.600291,0.591137,0.570185,0.559067
user_info,1.119651,0.604726,0.604878,0.591240,0.598462,0.607673,0.614567,0.597984,0.598545,0.629544,0.611326,0.595395,0.605898
user_log,1.108661,0.681693,0.695226,0.652143,0.630537,0.668254,0.692877,0.658842,0.641682,0.712748,0.704588,0.643950,0.616911


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.167025,0.637820,0.622240,0.612594,0.602888,0.583452,0.579135,0.574026,0.569727,0.668495,0.628498,0.612804,0.596927
list,1.151475,0.499394,0.499945,0.499991,0.500293,0.499151,0.500015,0.499762,0.499808,0.500249,0.499816,0.500263,0.500863
user_info,1.123632,0.605537,0.604988,0.591004,0.596489,0.608111,0.613386,0.596312,0.593785,0.629172,0.611191,0.595377,0.602798
user_log,1.122255,0.674021,0.681513,0.644018,0.619833,0.650880,0.669014,0.640308,0.621881,0.695420,0.687454,0.634597,0.604354


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.142210,0.634083,0.615181,0.608321,0.597484,0.590903,0.583143,0.579221,0.573297,0.669178,0.618840,0.608559,0.591370
list,1.157157,0.623609,0.612286,0.599127,0.587926,0.584215,0.585236,0.572121,0.561744,0.631417,0.605692,0.593941,0.581949
user_info,1.125399,0.601500,0.604869,0.591170,0.598697,0.601607,0.614773,0.598713,0.597784,0.623356,0.611688,0.594495,0.605012
user_log,1.100819,0.699980,0.708741,0.665576,0.639558,0.675847,0.703977,0.668767,0.649809,0.729600,0.712047,0.652886,0.618502


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.230989,0.651101,0.622593,0.611585,0.596444,0.603459,0.588435,0.580587,0.572376,0.704556,0.628166,0.612304,0.588429
list,1.134613,0.636266,0.621703,0.605817,0.594440,0.597266,0.600880,0.587887,0.576908,0.642617,0.605408,0.588278,0.577829
user_info,1.125101,0.605420,0.603102,0.591005,0.599191,0.608521,0.612548,0.596969,0.598773,0.625973,0.606858,0.592828,0.606204
user_log,1.152622,0.705698,0.704275,0.664168,0.638197,0.675480,0.688895,0.655241,0.637385,0.725917,0.706983,0.653594,0.619006


...

,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.466910,0.667824,0.632255,0.622259,0.605063,0.629112,0.602081,0.594656,0.582889,0.713066,0.640826,0.626820,0.601449
list,1.130563,0.630471,0.618098,0.600370,0.591766,0.584404,0.586237,0.573275,0.566702,0.650423,0.611510,0.588809,0.578765
user_info,1.119656,0.602102,0.605368,0.591229,0.597136,0.606676,0.614754,0.598126,0.597553,0.626791,0.612476,0.593281,0.601891
user_log,1.217929,0.715713,0.710130,0.674306,0.647318,0.690275,0.692552,0.661535,0.644559,0.736713,0.718651,0.674851,0.637805


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.496211,0.675681,0.640440,0.629297,0.612406,0.641853,0.612064,0.603755,0.591192,0.701656,0.647775,0.631930,0.608710
list,1.140787,0.635443,0.628021,0.611948,0.600269,0.587026,0.596402,0.584691,0.576676,0.647164,0.619086,0.599219,0.583745
user_info,1.120193,0.606468,0.607791,0.593636,0.598921,0.604772,0.613057,0.598681,0.598089,0.627574,0.614353,0.594884,0.602685
user_log,1.334778,0.720945,0.714699,0.680646,0.655209,0.688866,0.691011,0.659446,0.645569,0.737478,0.722888,0.683499,0.646119


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.589671,0.686926,0.645274,0.634829,0.617554,0.648351,0.613828,0.606029,0.594885,0.723637,0.652944,0.638158,0.612212
list,1.128466,0.643947,0.628220,0.615042,0.605859,0.595379,0.594590,0.584484,0.577505,0.669936,0.626080,0.609777,0.598435
user_info,1.122221,0.601003,0.598178,0.587251,0.595193,0.603049,0.605732,0.591607,0.592744,0.623246,0.603671,0.588862,0.599593
user_log,1.331703,0.724974,0.717376,0.685274,0.657797,0.696878,0.696231,0.664371,0.649162,0.752777,0.730670,0.692335,0.651483


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.654492,0.688283,0.653223,0.643913,0.625071,0.648345,0.621986,0.613531,0.600470,0.736473,0.662557,0.651131,0.623697
list,1.124423,0.638909,0.629824,0.614133,0.603719,0.591006,0.593055,0.581925,0.575360,0.661542,0.631807,0.610380,0.595406
user_info,1.120429,0.606413,0.605762,0.593034,0.599846,0.609510,0.615076,0.598918,0.598766,0.626692,0.611028,0.593763,0.602781
user_log,1.436945,0.726400,0.719175,0.689004,0.661596,0.700689,0.697749,0.667982,0.652126,0.749464,0.730622,0.696762,0.656366


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.713551,0.690082,0.660278,0.652447,0.631819,0.646859,0.623945,0.618629,0.604140,0.738106,0.674189,0.661389,0.631319
list,1.127511,0.640954,0.627704,0.612602,0.601511,0.599383,0.595839,0.584621,0.576018,0.657988,0.629090,0.608909,0.594685
user_info,1.118391,0.607518,0.605940,0.592596,0.598065,0.611665,0.613167,0.598249,0.598516,0.631934,0.613741,0.594435,0.601952
user_log,1.448180,0.725767,0.720757,0.690462,0.661835,0.699262,0.698224,0.667137,0.649860,0.748872,0.732580,0.698635,0.656709


### fit more

In [19]:
class MultiviewConfig():
    _instance = None
    def __new__(cls, *args, **kw):
        if cls._instance is None:
            cls._instance = object.__new__(cls, *args, **kw)
        return cls._instance
    
    def __init__(self):
        super().__init__()
        self.hidden = 256
#         self.app_list_net = torch.load('app_list_net.model.torch')
#         self.app_behave_net =  torch.load('app_list_net.model.torch')
#         self.user_net =  torch.load('user_net.model.torch')
        self.app_list_net = AppListNetwork(AppListConfig())
        self.app_behave_net = AppBehaveNetwork(AppbehaveConfig())
        self.user_net = UserNetwork(UserNetConfig())
        self.userlog_network = UserlogNetwork(Userlogconfig())
# torch.save(MultiviewConfig(), '../data_sortout/MultiviewConfig.torch')
logging.info('start training')
multiview_net = train(extend_train_ids, all_test_id, MaskMultiViewNetwork, MultiviewConfig())
logging.info('finish training')


2020-08-14 11:37:46,250 - INFO - start training


2020-08-14 12:05:12,328 - INFO - epoch : 0


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
test,1.079789,0.693936,0.696598,0.658135,0.644481,0.654081,0.671021,0.642842,0.632813,0.728141,0.702713,0.646250,0.628839
behave,1.100079,0.688637,0.697971,0.663319,0.649012,0.637654,0.663609,0.639115,0.628186,0.729069,0.705015,0.651977,0.633282
list,1.086328,0.698596,0.703014,0.665499,0.651853,0.644785,0.669418,0.643392,0.631071,0.739736,0.705008,0.648088,0.633191
user_info,1.080201,0.693936,0.696598,0.658135,0.644481,0.654081,0.671021,0.642842,0.632813,0.728141,0.702713,0.646250,0.628839
user_log,1.086474,0.701026,0.705381,0.663840,0.648937,0.665190,0.686086,0.653846,0.642287,0.728956,0.707739,0.649892,0.631158


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MaskMultiViewNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UserNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Dense. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GeLU. It won't be ch

2020-08-14 12:32:59,743 - INFO - epoch : 1


...

,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
test,2.035611,0.680612,0.658149,0.645980,0.632444,0.641870,0.621210,0.610691,0.602441,0.718249,0.679320,0.663330,0.638915
behave,2.589547,0.695185,0.664999,0.659390,0.642767,0.656802,0.628489,0.625320,0.615412,0.716126,0.678313,0.670685,0.642777
list,2.467130,0.695048,0.667398,0.663304,0.647318,0.654546,0.632431,0.628120,0.618847,0.727222,0.679099,0.677173,0.649392
user_info,2.035884,0.680612,0.658149,0.645980,0.632444,0.641870,0.621210,0.610691,0.602441,0.718249,0.679320,0.663330,0.638915
user_log,2.031931,0.698892,0.674153,0.661245,0.645551,0.665550,0.642921,0.630510,0.620398,0.727592,0.688809,0.674251,0.647939


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MaskMultiViewNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UserNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Dense. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GeLU. It won't be ch

2020-08-14 19:00:28,679 - INFO - epoch : 15


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
test,2.217268,0.675948,0.657566,0.643183,0.630803,0.637148,0.621052,0.608348,0.601418,0.711984,0.678465,0.661225,0.637723
behave,2.879688,0.692877,0.665820,0.658353,0.641712,0.653850,0.629438,0.624436,0.614892,0.711600,0.678458,0.669606,0.640992
list,2.732161,0.691681,0.667757,0.661493,0.645889,0.650191,0.633037,0.626555,0.617903,0.723200,0.678587,0.675352,0.647512
user_info,2.217303,0.675948,0.657566,0.643183,0.630803,0.637148,0.621052,0.608348,0.601418,0.711984,0.678465,0.661225,0.637723
user_log,2.214082,0.694850,0.673271,0.658035,0.643615,0.661589,0.642270,0.627520,0.618831,0.721988,0.687801,0.671711,0.646425


/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type MaskMultiViewNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UserNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Dense. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/huangweilin/anaconda3/envs/hyr/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GeLU. It won't be ch

In [22]:
for i in range(12):
    
    model = torch.load('model/multiview_net_extend.torch.model_%d' % i)
    
    install_behave_test = list(set(install_behave_set) & set(all_test_id))
    install_list_test = list(set(install_list_set) & set(all_test_id))
    user_info_test = list(set(user_info_set) & set(all_test_id))
    user_log_test = list(set(user_log_set) & set(all_test_id))
    USE_VIEW = "install_behave"
    behave_ret_dict = eval_data_single(model, install_behave_test)
    USE_VIEW = "install_list"
    list_ret_dict = eval_data_single(model, install_list_test)
    USE_VIEW = "user_info"
    userinfo_ret_dict = eval_data_single(model, user_info_test)
    USE_VIEW = "user_log"
    userlog_ret_dict = eval_data_single(model, user_log_test)
    
    df_ret = pd.DataFrame([
        behave_ret_dict, list_ret_dict, userinfo_ret_dict, userlog_ret_dict
    ], index = ['behave', 'list', 'user_info', 'user_log'])
    
    ipd.display(df_ret)

,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.151419,0.610516,0.606319,0.595627,0.586262,0.567335,0.567876,0.562448,0.557408,0.603451,0.609878,0.592159,0.577826
list,1.128143,0.648062,0.637915,0.621617,0.610092,0.602155,0.608384,0.594667,0.584611,0.652007,0.626155,0.607583,0.594486
user_info,1.124001,0.604086,0.597601,0.586197,0.593045,0.608615,0.608523,0.596375,0.594218,0.636272,0.600770,0.586177,0.600419
user_log,1.098608,0.676339,0.693397,0.647666,0.627751,0.661157,0.689369,0.654018,0.639672,0.710185,0.702257,0.637301,0.610199


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.151227,0.617536,0.611494,0.602950,0.593804,0.570432,0.572179,0.566781,0.562361,0.617862,0.611097,0.599664,0.585160
list,1.113510,0.661674,0.647118,0.632634,0.622122,0.610374,0.611661,0.596120,0.587032,0.671978,0.637058,0.625961,0.613340
user_info,1.123699,0.598060,0.595601,0.583030,0.591552,0.608187,0.611968,0.596180,0.594584,0.629910,0.600802,0.585447,0.598587
user_log,1.096344,0.678384,0.694382,0.650130,0.628433,0.668903,0.695791,0.661367,0.644693,0.711069,0.704623,0.642673,0.615034


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.182996,0.639590,0.617910,0.607496,0.600181,0.591447,0.577274,0.573484,0.571446,0.656641,0.621951,0.602134,0.589309
list,1.164627,0.647956,0.640234,0.622567,0.614827,0.599428,0.606866,0.589933,0.583380,0.654055,0.629542,0.611204,0.601941
user_info,1.123729,0.607023,0.603607,0.589297,0.596259,0.611861,0.615843,0.600290,0.599457,0.641474,0.609333,0.589922,0.599771
user_log,1.117667,0.681798,0.695947,0.653602,0.629759,0.655771,0.684244,0.650763,0.632187,0.719583,0.704421,0.645888,0.615581


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.205190,0.654479,0.622970,0.611132,0.599657,0.618181,0.595259,0.588168,0.582259,0.662277,0.617842,0.599482,0.581821
list,1.124230,0.655598,0.647769,0.632708,0.622389,0.599028,0.609425,0.596947,0.588727,0.668465,0.638322,0.622411,0.610196
user_info,1.130915,0.596780,0.595368,0.583295,0.591236,0.604643,0.608841,0.594923,0.593600,0.625190,0.600729,0.585060,0.597811
user_log,1.101114,0.686307,0.697898,0.655754,0.632237,0.664881,0.687565,0.654299,0.636252,0.727240,0.712470,0.655260,0.625287


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.323574,0.668098,0.631695,0.619802,0.604958,0.636997,0.606698,0.596684,0.587674,0.676468,0.629770,0.615157,0.592276
list,1.113482,0.661338,0.653923,0.638953,0.627534,0.610179,0.617187,0.603014,0.592312,0.667186,0.646290,0.631552,0.618584
user_info,1.122450,0.607189,0.605294,0.591807,0.600486,0.607968,0.613972,0.597799,0.598090,0.631502,0.609478,0.591358,0.603601
user_log,1.117952,0.678810,0.687440,0.648847,0.622595,0.655786,0.672586,0.640357,0.619424,0.702133,0.699044,0.649014,0.615884


...

,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.545263,0.685469,0.650033,0.641204,0.625241,0.649406,0.617340,0.610001,0.600388,0.710033,0.658226,0.646912,0.621198
list,1.120510,0.655633,0.648651,0.632778,0.622648,0.600986,0.611951,0.598320,0.589297,0.674136,0.641087,0.623717,0.611994
user_info,1.124772,0.604332,0.604345,0.590013,0.598447,0.607013,0.613457,0.596994,0.596771,0.636012,0.611461,0.593254,0.604310
user_log,1.151505,0.710105,0.709956,0.668563,0.642327,0.678815,0.692260,0.655598,0.636233,0.749635,0.722250,0.670179,0.635044


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.651665,0.684522,0.652058,0.640756,0.621417,0.649869,0.620076,0.609594,0.598690,0.703697,0.659861,0.647222,0.614907
list,1.117892,0.662496,0.654977,0.638972,0.627717,0.610818,0.618726,0.603789,0.593819,0.673309,0.648361,0.631248,0.617237
user_info,1.119867,0.603924,0.602922,0.591176,0.599555,0.608276,0.613267,0.599057,0.599044,0.630759,0.607676,0.593004,0.604213
user_log,1.184831,0.705383,0.704444,0.662025,0.634173,0.676924,0.689559,0.649727,0.628868,0.743714,0.715853,0.664836,0.627499


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.769170,0.688291,0.656882,0.646756,0.628611,0.654621,0.622262,0.613655,0.602821,0.711571,0.672098,0.658277,0.628116
list,1.114147,0.665337,0.655748,0.640327,0.627947,0.611024,0.617570,0.603359,0.592628,0.677132,0.650643,0.634148,0.618368
user_info,1.120544,0.603589,0.602415,0.589913,0.599653,0.608287,0.614019,0.598361,0.599465,0.626894,0.606947,0.592103,0.605026
user_log,1.211812,0.713222,0.708341,0.666432,0.641303,0.686297,0.690744,0.652614,0.634128,0.742840,0.720076,0.667730,0.633804


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,1.806189,0.679185,0.652010,0.645448,0.629034,0.646128,0.617986,0.611545,0.601913,0.698734,0.664661,0.656626,0.629021
list,1.114397,0.665028,0.655476,0.636834,0.624259,0.614555,0.621908,0.604243,0.592383,0.674199,0.647565,0.628559,0.613781
user_info,1.118068,0.607621,0.605797,0.591894,0.599979,0.610101,0.616789,0.600040,0.599273,0.632264,0.612453,0.595387,0.605978
user_log,1.222059,0.713870,0.711362,0.670169,0.645007,0.683102,0.692328,0.654027,0.634348,0.745180,0.722061,0.671225,0.636877


,loss,1m30+,2m30+,3m30+,4m30+,new_1m30+,new_2m30+,new_3m30+,new_4m30+,old_1m30+,old_2m30+,old_3m30+,old_4m30+
behave,2.022357,0.688254,0.658084,0.653647,0.637503,0.650943,0.624008,0.620653,0.611891,0.712784,0.666572,0.661479,0.633815
list,1.112212,0.665969,0.655127,0.635998,0.625450,0.612108,0.620803,0.603685,0.593170,0.684675,0.647444,0.625499,0.613043
user_info,1.118494,0.605787,0.606333,0.593214,0.600297,0.608677,0.616427,0.600532,0.599929,0.627061,0.611055,0.594894,0.603430
user_log,1.250506,0.712956,0.705891,0.665192,0.639798,0.678979,0.681776,0.644190,0.624504,0.748992,0.719541,0.668388,0.634297


### ret log

app list
1.111740	0.664095	0.657518	0.639428	0.630501

app behave
0.663820	0.638084	0.627594	0.623197	
0.624079	0.600459	0.592388	0.591546	
0.695377	0.652742	0.635372	0.625435

add time
0.682645	0.656571	0.646690	0.634663
0.636268	0.618727	0.611120	0.603158
0.732818	0.667061	0.652106	0.634041

add time & action
0.678883	0.653626	0.641810	0.628926	
0.636229	0.619752	0.609672	0.60488	
0.718630	0.658465	0.643400	0.619151

# debug code

In [ ]:
# class PositionalEncoding(nn.Module):
#     def __init__(self, config):
#         super(PositionalEncoding, self).__init__()
#         self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
#         max_len=config.max_position_embeddings
#         self.P=torch.zeros(1,max_len,config.hidden_size)
#         X=torch.arange(0,max_len).view(-1,1).float()/torch.pow(10000,torch.arange(0,config.hidden_size,2).float()/config.hidden_size)
#         self.P[:,:,0::2]=torch.sin(X.float())
#         self.P[:,:,1::2]=torch.cos(X.float())
#         self.Dropout = nn.Dropout(config.hidden_dropout_prob)
#         self.layerNorm = BertLayerNorm(config.hidden_size, eps=config.layer_norm_eps)
#     def forward(self,X,position_ids=None):
#         input_shape = X.size()[:-1]
#         seq_length = input_shape[1]
#         device = X.device
#         inputs_embeds=X
#         if position_ids is None:
#             position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
#             position_ids = position_ids.unsqueeze(0).expand(input_shape)
#         position_embeddings = self.position_embeddings(position_ids)

#         embeddings = inputs_embeds + position_embeddings
#         embeddings = self.layerNorm(embeddings)
#         embeddings = self.Dropout(embeddings)
#         return embeddings
# class Transformer(nn.Module):
#     def __init__(self,config):
#         super(Transformer,self).__init__()
#         self.config=config
#         self.Encoder=BertEncoder(config=config)
#         self.P=PositionalEncoding(config)
#         self.rnns=nn.ModuleList()
#         for i in range(1):
#             hidden_size=config.hidden_size
#             self.rnns.append(nn.GRU(input_size=config.hidden_size,hidden_size=config.hidden_size,num_layers=1,\
#                         bidirectional=True,batch_first=True))
#         self.one=nn.Parameter(torch.tensor([1]).float())
#         for n,e in self.Encoder.named_modules():
#             self._init_weights(e)
#         for n,e in self.P.named_modules():
#             self._init_weights(e)
            
#     def _init_weights(self, module):
#         """ Initialize the weights """
#         if isinstance(module, (nn.Linear, nn.Embedding)):
#             # Slightly different from the TF version which uses truncated_normal for initialization
#             # cf https://github.com/pytorch/pytorch/pull/5617
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#         elif isinstance(module, BertLayerNorm):
#             module.bias.data.zero_()
#             module.weight.data.fill_(1.0)
#         if isinstance(module, nn.Linear) and module.bias is not None:
#             module.bias.data.zero_()
#     def make_mask(self,X,valid_len):
#             shape=X.shape
#             if valid_len.dim()==1:
#                 valid_len=valid_len.view(-1,1).repeat(1,shape[1])
#             mask=(torch.arange(0,X.shape[1]).repeat(X.shape[0],1).to(X.device)<valid_len).float()
#             return mask
#     def forward(self, X, length, encoder_hidden_states=None,encoder_extended_attention_mask=None):

#         attention_mask=self.make_mask(X,length)
#         embedding_output=X
#         if attention_mask.dim() == 3:
#             extended_attention_mask = attention_mask[:, None, :, :]
#         elif attention_mask.dim() == 2:
#             extended_attention_mask = attention_mask[:, None, None, :]
#         extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)  # fp16 compatibility
#         extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        
#         #make head mask
#         head_mask = [None] * self.config.num_hidden_layers
#         outputs=self.Encoder(  embedding_output,
#                 attention_mask=extended_attention_mask,
#                 head_mask=head_mask,
#                 encoder_hidden_states=encoder_hidden_states,
#                 encoder_attention_mask=encoder_extended_attention_mask,)
#         return outputs[0]
    
# class LayerNorm(nn.Module):
#     def __init__(self,features,eps=1e-6):
#         super(LayerNorm,self).__init__()
#         self.gamma=nn.Parameter(torch.ones(features))
#         self.beta=nn.Parameter(torch.zeros(features))
#         self.eps=eps
#     def forward(self,X):
#         mean=X.mean(-1,keepdim=True)
#         std=X.std(-1,keepdim=True)
#         return self.gamma*(X-mean)/(std+self.eps)+self.beta

    
# config = {
#   "attention_probs_dropout_prob": 0, 
#   "directionality": "bidi", 
#   "hidden_act": "gelu", 
#   "hidden_dropout_prob": 0, 
#   "hidden_size": 4, 
#   "initializer_range": 0.02, 
#   "intermediate_size": 840, 
#   "max_position_embeddings": 128, 
#   "num_attention_heads": 1, 
#   "num_hidden_layers": 1, 
#   "type_vocab_size": 2, 
#   "vocab_size": 21128,
# }
# config=BertConfig(**config, output_hidden_states=True, output_attentions=True)

# X = torch.tensor([
#     [
#         [1, 2, 3, 4],
#         [1, 2, 3, 4],
#         [0, 0, 0, 0],
#     ],
#     [
#         [2, 2, 3, 4],
#         [1, 2, 3, 4],
#         [2, 6, 0, 0],
#     ]
# ]).float()

# Length = torch.tensor([2, 3]).long()
# model = Transformer(config)

# model(X, Length)